# 쓰레드는 무엇이며, 왜 이용하는가 ?

쓰레드는 세미(semi)프로세스, 혹은 Light Weight 프로세스라고 불리우며, 여러개의 클라이언트를 처리하는 서버/클라이언트 모델의 서버프로그래밍 작업을 위해서 주로 사용된다. 비슷한 일을 하는 fork() 에 비해서 빠른 프로세스 생성 능력과, 적은 메모리를 사용하는게 Light Weight 프로세스라고 불리우는 이유이다.

스레드는 **자기자신의 스택메모리영역**을 가지고, 코드의 조각을 실행한다. **실(real) 프로세스 와는 달리 쓰레드는 다른 형제 쓰레드들과 메모리를 공유**하게 된다.(보통 프로세스는 자기자신만의 메모리영역을 가진다). 이렇듯 **전역 메모리를 공유하게 되므로 fork 방식에 비해서 좀더 작은 메모리를 소비**하게 된다

# thread 프로그래밍의 주요 요소들

## 병렬 프로그래밍
병렬프로그램의 작성에는 고려해야할 여러가지 사항들이 있다. 때때로 이들은 프로그램의 작성을 매우 어렵게 만들기도 한다
로드 밸런싱
* **Data dependencies**
* **동기화 그리고 race conditions**
* **메모리 이슈**
* **IO 이슈**
* **프로그램의 복잡도 증가**
* **개발시간과 비용의 증가**

일반적으로 사용되는 쓰레드 프로그램 모델은 다음과 같다.
* **Manager/worker**
<pre>boss/worker 모델이라고 부르기도 한다. 일반적으로 Manager 쓰레드가 모든 입력을 제어하고 각 쓰레드에 작업을 배분한다. server & client 모델을 사용하는 네트워크 프로그램의 제작에 널리 사용된다.</pre>

## shared memory 모델
**모든 쓰레드가 전역 메모리공간을 공유하는 방식**이다. 프로그래머는 전역 메모리 공간에 대한 Access 동기화에 특히 신경을 써줘야 한다.

## thread safeness
**쓰레드는 공유하는 메모리공간에 대해서 서로 제어권을 얻기위해서 경쟁하는 상태**에 놓일 수 있다. 이들 **쓰레드가 자원을 놓고 경쟁하는 것을 제어하지 않으면 쓰레드가 안전 - safeness - 하지 않은 상태**에 놓이게 된다.


# POSIX thread

## 쓰레드의 생성과 종료
쓰레드를 이용한 프로그램은 기본적으로 아래와 같은 순서로 작동하게 된다.
![Alt text](./thread/thread_1.png)

아래는 쓰레드 프로그램의 가장 간단한 예이다.
thread_1.c
~~~
#include <stdio.h> 
#include <unistd.h> 
#include <pthread.h> 

void* do_loop(void *data)
{
    int i;

    int me = *((int *)data);
    for (i = 0; i < 10; i++)
    {
        printf("%d - Got %d\n", me, i);
        sleep(1);
    }
}

int main()
{
    int       thr_id;
    pthread_t p_thread[3];
    int status;
    int a = 1;
    int b = 2;      
    int c = 3;      

    thr_id = pthread_create(&p_thread[0], NULL, do_loop, (void *)&a);
    thr_id = pthread_create(&p_thread[1], NULL, do_loop, (void *)&b);
    thr_id = pthread_create(&p_thread[2], NULL, do_loop, (void *)&c);

    pthread_join(p_thread[0], (void **) &status);
    pthread_join(p_thread[1], (void **) &status);
    pthread_join(p_thread[2], (void **) &status);

    printf("programing is end\n");
    return 0;
}
~~~


위의 프로그램을 컴파일 시키기 위해선 pthread 라이브러리를 링크시켜줘야 한다.

 **gcc -o thread thread.c -lpthread**

최초에 main() 쓰레드가 시작되고 나서 pthread_create 를 이용해서 3개의 쓰레드를 생성 시켯다. 각각의 쓰레드는 do_loop 코드를 실행한다. 쓰레드가 모든 작업을 마쳤다면, pthread_join 을 이용해서 다른 쓰레드가 종료될때까지 기다리고, 모든 쓰레드가 종료되었다면, main()쓰레드가 종료되고 프로세스는 완전히 끝나게 된다.
쓰레드의 생성은 pthread_create()를 호출함으로써 이루어진다. 첫번째 아규먼트는 pthread_t 데이타 구조체에 대한 포인터를 돌려주는데, 쓰레드에 대한 지시값이 들어 있다. 각각의 쓰레드는 각각의 유일한 pthread_t 를 가지고 있어야만 한다. 위의 프로그램에서 우리는 각각의 쓰레드가 유일한 p_thread 를 가지도록 하기 위해서 생성할 쓰레드의 수만큼(3)을 배열로 만들었다. 2번째 아규먼트는 쓰레드가 만들어질때의 타입이다.
(스케쥴링 우선순위 같은). 보통은 NULL 값을 사용한다. 쓰레드 타입에 대한 내용은 pthread_attr_init(3) 을 참조하기 바란다. 3번째 아규먼트가 바로 쓰레드가 실행할 코드이다. 4번째 아규먼트는 쓰레드에 넘겨주고 싶은 값을 명시해주면 된다. 여기에서는 각 쓰레드에 번호를 부여하기위한 int 값을 넘겼다.
각 쓰레드는 1부터 10까지 증가 시킨다음에 쓰레드를 종료하도록 되어 있다.
그동안 메인 쓰레드는 pthread_join 을 호출하여서 각각의 쓰레드가 종료할때까지 기다린다. 3개의 쓰레드가 모두 종료가 된다면 메인 쓰레드는 "programing is end" 메시지를 출력하고 프로그램을 완전히 종료하게 될것이다.
pthread_join 은 fork 의 wait(2) 와 비슷하다고 볼수 있다. fork 에서도 자식프로세스가 모두죽고 나서 부모프로세스가 죽어야 하듯이(예외를 만들수도 있지만), 쓰레드도 모든 생성된 쓰레드가 종료된 다음에 메인 쓰레드가 종료되어야 한다.
pthread_join 을 사용하게 되면 메인 쓰레드는 pthread_join 에 명시된 쓰레드가 종료할때까지 잠자면서(sleep)기다리게 된다. 이는 모든 쓰레드가 종료하기 전에 부모쓰레드가 종료하는 사태를 막기 위해서 사용된다. **하나의 쓰레드가 모든일을 종료하고, pthread_join 을 깨우게 되면, 쓰레드가 가지고 있던 자원들을 모두 되돌려주게 된다 (free). 만약 실행되고 있는 쓰레드를 즉시 중지하길 원한다면 pthread_cancel() 과 pthread_testcancel()을 사용하면 된다.**

## 공용으로 사용되는 자원의 동기화
기본적으로 하나의 쓰레드가 하나의 자원에 접근하고 있을때, 다른 쓰레드는 그 자원에 대한 이전 쓰레드의 작업이 모두 끝나기전엔 접근하면 안될것이다.
이런한 공유되는 자원에 대한 접근제어는 **IPC 설비의 세마포어와 매우 비슷한 점**이 있다. 쓰레드에서는 이러한 공유되는 자원의 접근 제얼르 위해서 **Mutexe** 라는 것을 제공

# 기본 쓰레드 함수

## pthread_create
**int pthread_create(pthread_t *thread, const pthread_attr_t *attr, void *(*start_routine)(void *), void *arg);**

쓰레드 생성을 위해서 사용한다. 첫번째 아규먼트인 thread 는 쓰레드가 성공적으로 생성되었을때 생성된 쓰레드를 식별하기 위해서 사용되는 쓰레드 식별자이다. 두번째 아규먼트인 attr 은 쓰레드 특성을 지정하기 위해서 사용하며, 기본 쓰레드 특성을 이용하고자 할경우에 NULL 을 사용한다. 3번째 아규먼트인 start_routine는 분기시켜서 실행할 쓰레드 함수이며, 4번째 아규먼는인 arg는 쓰레드 함수의 인자이다.
성공적으로 생성될경우 0을 리턴한다.

## pthread_join
**int pthread_join(pthread_t th, void **thread_return);**

**실행된 쓰레드에 대해서는 pthread_join 등의 함수를 이용해서 쓰레드 종료때까지 기다려줘야 한다. ptherad_join 은 일종의 fork 의 wait 와 비슷하게 작동하며, 쓰레드자원을 해제 시켜준다**.

첫번째 아규먼트 th는 기다릴(join)할 쓰레드 식별자이며, 두번째 아규먼트 thread_return은 쓰레드의 리턴(return) 값이다. thread_return 이 NULL 이 아닐경우 포인터로 쓰레드 리턴 값을 받아올수 있다.

## pthread_detach
**int pthread_detach(pthread_t th);**

detach 는 "떼어내다" 라는 뜻을 가지며 **main 쓰레드에서 pthread_create 를 이용해 생성된 쓰레드를 분리시킨다. 이 함수는 식별번호th인 쓰레드를 detach 시키는데, detach 되었을경우 해당(detach 된) 쓰레드가 종료될경우 pthread_join 을 호출하지 않더라도 즉시 모든 자원이 해제(free)** 된다.
여기에서는 pthread_create 호출후 detach 하는 방법을 설명하고 있는데, pthread_create 호출시에 쓰레드가 detach 되도록 할수도 있다.

detatach 를 했을경우 프로세스의 메모리 사용율과 detache 를 주석 처리했을경우의 메모리 사용율의 변화를 서로 비교해보면 되는데, detach 를 사용하지 않았을경우 t_function 이 종료가 되더라도 자원이 해제되지 않음을 볼수 있을것이다

## pthread_exit
**void pthread_exit(void *retval);**

**pthread_exit 는 현재 실행중인 쓰레드를 종료시키고자 할때 사용한다. 만약 pthread_cleanup_push 가 정의되어 있다면, pthread_exit 가 호출될경우 cleanup handler 가 호출된다. 보통 이 cleanup handler 은 메모리를 정리하는 등의 일을 하게 된다**.

## pthread_cleanup_push
**void pthrad_cleanup_push(void (*routine) (void *), void *arg);**

**이것은 cleanup handlers 를 인스톨하기 위해서 사용된다. pthread_exit(3) 가 호출되어서 쓰레드가 종료될때 pthread_cleanup_push 에 의해서 인스톨된 함수가 호출된다. routine이 쓰레드가 종료될때 호출되는 함수**이다. arg는 아규먼트이다.
**cleanup handlers 는 주로 자원을 되돌려주거나, mutex 잠금등의 해제를 위한 용도로 사용된다. 만약 mutex 영역에서 pthread_exit 가 호출되어 버릴경우 다른쓰레드에서 영원히 block 될수 있기 때문이다. 또한 malloc 으로 할당받은 메모리, 열린 파일지정자를 닫기 위해서도 사용**한다.

## pthread_cleanup_pop
**void pthread_cleanup_pop(int execute);**

pthread_cleanup_push 와 함께 사용되며, install 된 cleanup handler 을 제거하기 위해서 사용된다.
**만약 execute 가 0 이라면, pthread_cleanup_push 에 의해 인스톨된 cleanup handler 를 (실행시키지 않고)삭제만 시킨다. 0 이 아닌 숫자라면 cleanup handler 을 실행시키고 삭제 된다**

## pthread_self
**pthread_t pthread_self(void);**

pthread_self를 호출하는 현재 쓰래드의 쓰레드식별자를 되돌려준다.

# 쓰레드 동기화 함수

##  pthread_mutex_init
**int pthread_mutex_init(pthread_mutex_t * mutex, const pthread_mutex_attr *attr); **

mutex 는 **여러개의 쓰레드가 공유하는 데이타를 보호하기 위해서 사용되는 도구로써, 보호하고자 하는 데이타를 다루는 코드영역을 단지 한번에 하나의 쓰레드만 실행가능 하도록 하는 방법으로 공유되는 데이타를 보호**한다. 이러한 **코드영역(하나의 쓰레드만 점유가능한)을 critical section** 이라고 하며, mutex 관련 API 를 이용해서 관리할수 있다.
pthread_mutex_init 는 mutex 객체를 초기화 시키기 위해서 사용한다. 첫번째 인자로 주어지는 mutex 객체 mutex를 초기화시키며, 두번째 인자인 attr 를 이용해서 mutex 특성을 변경할수 있다. 기본 mutex 특성을 이용하기 원한다면 NULL 을 사용하면 된다.
**mutex 특성(종류) 에는 "fast", "recursive", "error checking" 의 종류가 있으며, 기본으로 "fast" 가 사용**된다.

~~~
// 뮤텍스 객체 선언
pthread_mutex_t mutex_lock;
...
void *t_function()
{
    pthread_mutex_lock(&mutex_lock);
    // critical section
    pthread_mutex_unlock(&mutex_lock);
}
int main()
{
    pthread_t p_thread;
    int state;
    // 뮤텍스 객체 초기화, 기본 특성으로 초기화 했음
    pthread_mutex_init(&mutex_lock, NULL);
    pthread_create(&p_thread, NULL, t_function, (void *)&a);
    ...
    pthread_join(&p_thread, (void **)&status);
}
~~~

## pthread_mutex_destroy
**int pthread_mutex_destroy(pthread_mutex_t *mutex);**

자로 주어진 뮤텍스 객체 mutex 를 제거하기 위해서 사용된다. mutex 는 pthread_mutex_init()함수를 이용해서 생성된 뮤텍스 객체이다.
pthread_mutex_destroy 를 이용해서 제대로 **mutex 를 삭제하려면 이 mutex 는 반드시 unlock 상태**이여야 한다.

## pthread_mutex_lock
**int pthread_mutex_lock(pthread_mutex_t *mutex);**

**pthread_mutex_lock 는 critcal section 에 들어가기 위해서 mutex lock 을 요청한다. 만약 이미 다른 쓰레드에서 mutex lock 를 얻어서 사용하고 있다면 다른 쓰레드에서 mutex lock(뮤텍스 잠금) 을 해제할때까지(사용할수 있을때까지) 블럭** 된다.
만약 다른 어떤 쓰레드에서도 mutex lock 을 사용하고 있지 않다면, 즉시 mutex lock 을 얻을수 있게 되고 critcal section 에 진입하게 된다. critcal section 에서의 모든 작업을 마쳐서 사용하고 있는 mutex lock 이 더이상 필요 없다면 pthread_mutex_unlock 를 호출해서 mtuex lock 를 되돌려준다.

## pthread_mutex_unlock
**int pthread_mutex_unlock(pthread_mutex_t *mutex);**

critical section 에서의 모든 작업을 마치고 mutex lock 을 돌려주기 위해서 사용한다. **pthread_mutex_unlock 를 이용해서 mutex lock 를 되돌려주면 다른 쓰레드에서 mutex lock 를 얻을수 있는 상태**가 된다.

## pthread_cond_init
**int pthread_cond_init(pthread_cond_t *cond, const pthread_cond_attr *attr);**

**pthread_cond_init는 조견변수 (condition variable)cond를 초기화하기 위해서 사용**한다. attr 를 이용해서 조건변수의 특성을 변경할수 있으며, NULL 을 줄경우 기본특성으로 초기화된다.
**조건변수 cond는 상수 PTHREAD_COND_INITIALIZER 을 이용해서도 초기**화 할수 있다. 즉 다음과 같은 2가지 초기화 방법이 존재한다.

**pthread_cond_t cond = PTHREAD_COND_INITIALIZER;**
or
**pthread_cond_init(&cond, NULL);**

## pthread_cond_signal
**int pthread_cond_signal(pthread_cond_t *cond);**

**조건변수 cond에 시그날을 보낸다. 시그날을 보낼경우 cond에서 기다리는(wait) 쓰레드가 있다면 쓰레드를 깨우게 된다(봉쇄가 풀림). 만약 조건변수 cond를 기다리는 쓰레드가 없다면, 아무런 일도 일어나지 않게되며, 여러개의 쓰레드가 기다리고 있다면 그중 하나의 쓰레드에게만 전달된다. 이때 어떤 쓰레드에게 신호가 전달될지는 알수 없다.**

## pthread_cond_boradcast
**int pthread_cond_broadcast(pthread_cond_t *cond);**

**조건변수 cond에서 기다리는(wait) 모든 쓰레드에게 신호를 보내서, 깨운다는 점을 제외하고는 pthread_cond_signal과 동일하게 작동**한다.

## pthread_cond_wait
**int pthread_cond_wait(pthread_cond_t cond, pthread_mutex_t *mutex); **

**조건변수 cond를 통해서 신호가 전달될때까지 블럭된다. 만약 신호가 전달되지 않는다면 영원히 블럭될수도 있다. pthread_cond_wait는 블럭되기 전에 mutex 잠금을 자동으로 되돌려준다.**

## pthread_cond_timewait
**int pthread_cond_timedwait(pthread_cont_t *cond, pthread_mutex_t *mutex, const struct timespec *abstime);**

**조건변수 cond를 통해서 신호가 전달될때까지 블럭되며 자동으로 mutex을 돌려주는 점에서는 pthread_cond_wait와 동일하다. 그러나 시간체크가 가능해서 abstime시간동안 신호가 도착하지 않는다면 error 를 발생하면서 리턴한다**. 이때 리턴값은 ETIMEDOUT 이다. **errno 가 세팅되는게 아닌, 리턴값으로 에러가 넘어오는것에 주의**해야 한다.
또한 **pthread_cond_timedwait함수는 다른 signal 에 의해서 interrupted 될수 있으며 이때 EINTR 을 리턴**한다. 이 함수를 쓸때는 interrupted 상황에 대한 처리를 해주어야 한다.

## pthread_cond_destroy
**int pthread_cond_destroy(pthread_cond_t *cond);**

**pthread_cond_init를 통해서 생성한 조건변수cond에 대한 자원을 해제한다. destroy 함수를 호출하기 전에 어떤 쓰레드도 cond에서의 시그널을 기다리지 않는걸 확인해야 한다.** 만약 cond 시그널을 기다리는 쓰레드가 존재한다면 이 함수는 실패하고 EBUSY 를 리턴한다.

## 예제코드
~~~
이번장에서 설명한 쓰레드 동기화 관련 함수의 이해를 돕기 위해서 간단한 예제를 준비했다. 설명은 주석으로 대신한다.

예제 : pthrad_sync_api.c

#include <pthread.h>
#include <unistd.h>
#include <stdio.h>
#include <stdlib.h>
#include <vector>
#include <iostream>

using namespace std;

void *ping(void *);
void *pong(void *);

pthread_mutex_t sync_mutex;
pthread_cond_t  sync_cond;

pthread_mutex_t gmutex = PTHREAD_MUTEX_INITIALIZER;
pthread_cond_t  gcond  = PTHREAD_COND_INITIALIZER;

int main()
{
    vector<void *(*)(void *)> thread_list;
    vector<pthread_t> tident(10); 
    int thresult;
    int status;
    int i;

    pthread_mutex_init(&sync_mutex, NULL);
    pthread_cond_init(&sync_cond, NULL);

    thread_list.push_back(pong);
    thread_list.push_back(ping);

    for(i = 0; i < thread_list.size(); i++ )
    {
        pthread_mutex_lock(&sync_mutex);
        if (pthread_create(&tident[i], NULL, thread_list[i], (void *)NULL) <0)
        {
            perror("error:");
            exit(0);
        }
        pthread_cond_wait(&sync_cond, &sync_mutex);
        pthread_mutex_unlock(&sync_mutex);
    }
    for (i = 0; i < tident.size(); i++)
    {
        pthread_join(tident[i], (void **)&status);
    }
}

void *ping(void *data)
{
    int i=0;
    pthread_mutex_lock(&sync_mutex);
    pthread_cond_signal(&sync_cond);
    pthread_mutex_unlock(&sync_mutex);
    while(1)
    {
        pthread_mutex_lock(&gmutex);
        printf("%d : ping\n", i);
        pthread_cond_signal(&gcond);
        pthread_cond_wait(&gcond, &gmutex);
        pthread_mutex_unlock(&gmutex);
        usleep(random()%100);
        i++;
    }
}

void *pong(void *data)
{
    int i = 0;
    pthread_mutex_lock(&sync_mutex);
    sleep(1);
    pthread_cond_signal(&sync_cond);
    pthread_mutex_unlock(&sync_mutex);
    while(1)
    {
        pthread_mutex_lock(&gmutex);
        pthread_cond_wait(&gcond, &gmutex);
        printf("%d : pong\n", i);
        pthread_cond_signal(&gcond);
        pthread_mutex_unlock(&gmutex);
        i++;
    }
}
~~~

위의 예제는 ping&pong 프로그램으로 ping 쓰레드와 pong 쓰레드가 각각 번갈아가면서 "ping", "pong" 을 날리는 프로그램이다. 2개의 영역에 걸쳐서 크리티컬섹션이 지정되어 있으며 각 크리티컬섹션안에는 쓰레드 동기화를 위해서 ptread_cond_signal 이 쓰여지고 있다.

위의 코드는 기본적으로 pong 쓰레드가 먼저 시그널을 대기하고 있다가 그 후 ping 쓰레드가 진입해서 "ping"을 날리고 시그널을 발생시키면 "pong" 메시지를 발생시키도록 되어 있다. 그렇다면 while 문에 있는 크리티컬 섹션에 반드시 pong 쓰레드가 먼저 진입할수 있도록 만들어줘야 할것이다. 그래서 위의 코드에서는 pong 쓰레드를 먼저 생성시켰다. 그러나 이것만으로는 충분하지 않다. 예를들어서 pong 쓰레드에서 크리티컬섹션에 들어가기 위해서 어떤 부가적인 작업이 있다고 했을때(메모리초기화, 기타 다른 함수 호출과 같은, 위에서는 sleep 으로 대신했다), 우리가 의도했던 바와는 다르게 ping 가 먼저 크리티컬섹션에 진입할수도 있다. 이럴경우 2개의 쓰레드는 교착상태에 빠지게 된다.

ping 쓰레드가 크리티컬섹션에 먼저 진입했을경우 ping 쓰레드는 "ping" 출력시키고 시그널을 발생시킬 것이고 pong 쓰레드가 "pong"를 출력시키고 시그널을 발생시킬때까지 시그널대기 하게 된다. ping 쓰레드가 시그널대기 하게 되면, 크리티컬섹션에 대한 뮤텍스 잠금이 해제됨으로 뒤늦게 크리티컬섹셔네 진입을 시도하던 pong 가 크리티컬섹션에 진입하고 ping 쓰레드에서부터 신호가 있는지 기다리게 될것이다. 그러나 ping 쓰레드는 이미 신호를 날려버렸음으로, pong 쓰레드는 결코 도착하지 않을 신호를 기다리며 영원히 시그널대기 하게 될것이다. 이런식으로 2개의 쓰레드는 교착상태에 빠져 버린다.

이 문제는 쓰레드간 동기화를 이용해서 해결할수 있으며, **위 코드에서는 mutex 잠금과, 조건변수를 이용해서 해결**하고 있다. 물론 쓰레드간 동기화를 위해서 사용할수 있는 원시적인 방법으로 sleep 나 usleep 같은 함수를 호출하는 방법도 있긴 하지만, ping 쓰레드에서 크리티컬 섹션에 진입하기전 1초 정도 sleep 을 주는 식으로 사용가능하지만 추천할만하진 않다. (간혹 간단하게 사용할수는 으며, 가장 확실한 방법을 제공해 주기도 한다)

# Thread Attribute 함수

## pthread_attr_init
**int pthread_attr_init(pthread_attr_t *attr);**

pthread_attr_init는 thread attribute 객체인 attr을 디폴트 값으로 초기화 시킨다.
성공할경우 0을 돌려주고 실패할경우 -1 을 되돌려준다.

## pthread_attr_distroy
**int pthread_attr_destroy(pthread_attr_t *attr);**

pthread_attr_init에 의해 생성된 thread attribute 객체인 attr을 제거한다. 제거된 attr 을 다시 사용하기 위해서는 pthread_attr_init를 이용해서 다시 초기화 해주어야 한다.

## pthread_attr_getscope
**int pthread_attr_getscope(const pthread_attr_t *attr, int *scope);**

쓰레드가 어떤 영역(scope)에서 다루어지고 있는지를 얻어오기 위해서 사용된다. PTHREAD_SCOPE_SYSTEM과 PTHREAD_SCOPE_PROCESS 의 2가지 영역중에 선택할수 있다. SYSTEM 영역 쓰레드는 user 모드 쓰레드라고 불리우며, PROCESS 쓰레드는 커널모드 쓰레드라고 불리운다. 리눅스의 경우 유저모드 쓰레드인데, 즉 커널에서 쓰레드를 스케쥴링하는 방식이 아닌 쓰레드 라이브러리를 통해서 쓰레드를 스케쥴링 하는 방식을 사용한다.

~~~
#include <pthread.h>
#include <stdlib.h>
#include <stdio.h>


int main()
{
    pthread_attr_t pattr;
    int scope;

    pthread_attr_init(&pattr);

    pthread_attr_getscope(&pattr, &scope);
    if (scope == PTHREAD_SCOPE_SYSTEM)
    {
        printf("user mode thread\n");
    }
    else if (scope ==  PTHREAD_SCOPE_PROCESS)
    {
        printf("Kernel mode thread\n");
    }

    return 1;
}
~~~

위 프로그램을 컴파일한후 Linux 에서 실행시키면 "user mode thread"를 출력하고 솔라리스 상에서 실행시키면 "kernel mode thread"를 출력한다.

## pthread_attr_setscope
**int pthread_attr_setscope(pthread_attr_t *attr, int scope);**

쓰레드가 어떤 영역(scope)에서 작동하게 할것인지 결정하기 위해서 사용한다. 리눅스의 경우 Kernel mode 쓰레드를 지원하지 않음으로 오직 PTHREAD_SCOPE_SYSTEM 만을 선택할수 있다. 반면 솔라리스는 유저모드와 커널모드중 선택이 가능하다.

## pthread_attr_getdetachstate
**int pthread_attr_getdetachstate(pthread_attr_t *attr, int detachstate);**

쓰레드가 join 가능한 상태(PTHREAD_CREATE_JOINABLE) 인지 detached 상태인지 (PTHREAD_CREATE_DETACHED) 인지를 알아낸다. 알아낸 값은 아규먼트 detachstate 에 저장된다.
기본은 PTHREAD_CREATE_JOINABLE 이며, pthread_detach를 이용해서 생성된 쓰레드를 detach 상태로 만들었을경우 또는 pthread_attr_setdetachstate함수를 이용해서 쓰레드를 detache 상태로 변경시켰을경우 PTHREAD_CREATE_DETACHED 상태가 된다.

~~~
#include <pthread.h>
#include <stdlib.h>
#include <stdio.h>

pthread_attr_t attr;
void *test(void *a)
{
    int policy;
    printf("Thread Create\n");
    pthread_attr_getdetachstate(&attr, &policy);
    if (policy == PTHREAD_CREATE_JOINABLE)
    {
        printf ("Join able\n");
    }
    else if (policy == PTHREAD_CREATE_DETACHED)
    {
        printf ("Detache\n");
    }
}
int main()
{
    int status;
    pthread_t p_thread;
    pthread_attr_init(&attr);
    if (pthread_create(&p_thread, NULL, test, (void *)NULL) < 0)
    {
        exit(0);
    }

    pthread_join(p_thread, (void **)&status);
}
~~~

위의 프로그램을 실행시키면 분명 "Join able"를 출력할것이다.

## pthread_attr_setdetachstate
**int  pthread_attr_setdetachstate(pthread_attr_t *attr, int detachstate);**

쓰레드의 상태를 PTHREAD_CREATE_JOINABLE 혹은 PTHREAD_CREATE_DETACHED 상태로 변경시키기 위해서 사용된다. 아래와 같은 방법으로 사용하면 된다.

pthread_attr_t attr;
...
// JOINABLE 상태로 변경하고자 할때 
pthread_attr_setdetachstate(&attr, PTHREAD_CREATE_JOINABLE);
// DETACHED 상태로 변경하고자 할때
pthread_attr_setdetachstate(&attr, PTHREAD_CREATE_DETACHED);

# 쓰레드 시그널 관련

## pthread_sigmask
**int pthread_sigmask(int how, const sigset_t *newmask, sigset_t *oldmask);**

쓰레드에서 시그널은 서로 공유된다. 그런이유로 만약 프로세스에 시그널이 전달되면 프로세스가 생성된 모든 쓰레드로 시그널이 전달된다. 그러나 특정 쓰레드만 시그널을 받도록 하고 싶을 때가 있을 것이다. 이경우 이 함수를 이용하면 된다.

## pthread_kill
**int pthread_kill(pthread_t thread, int signo);**

쓰레드 식별번호 thread로 signo 번호의 시그널을 전달한다.

## sigwait
**int sigwait(const sigset_t *set, int *sig);**

시그널 전달을 동기적으로 기다린다.

# 쓰레드 취소

## pthread_cancel
**int pthread_cancel(pthread_t thread);**

## pthread_setcancelstate
**int pthread_setcancelstate(int state, int *oldstate);**

## pthread_setcancelstate
**int pthread_setcancelstate(int state, int *oldstate);**

## pthread_setcanceltype
**int pthread_setcanceltype(int type, int *oldtype);**

## pthread_testcancel
**void pthread_testcancel(void);**

# 공유 자원
접근 제어가 필요한 공간"에는 보호 해야 할 공유 자원이 놓인다. 보호 해야할 공유 자원이 있는 공간을 임계 영역이라고 한다. 시간 제어는 해당 임계 영역에 동 시간에 단지 하나의 쓰레드만 접근하도록 제한 하는 방식으로 이루어진다.
임계영역에 들어가기 위한 하나의 키를 가지고 경쟁하는 것으로 이해하면 된다. 임계영역에 들어가기 위한 키는 단지 하나 밖에 없으므로 어떤 스레드가 키를 얻어서 임계영역에 진입하면, 다른 스레드는 키를 얻을 때까지 - 앞서 임계영역에 진입한 프로세스가 키를 되돌려줄 때까지 - 기다려야 한다

![Alt text](./thread/thread_2.png)

## Mutex
뮤텍스는 pthread에서 제공하는 동기화 매커니즘으로 공유 자원 공간에 대한 접근 시간 제어로 동기화를 달성한다. 기본적인 매커니즘은 세마포어와 비슷하다. 특히 POSIX 세마포어와 비슷하며, 동기화 매커니즘으로 뮤텍스 대신 세마포어를 사용할 수도 있다. 동기화 매커니즘의 핵심은 상호 배제로 다음과 같이 달성 한다.

**상호 배제는 잠금 형식**으로 이루어진다. 쓰레드는 잠금 v를 얻어야 임계 영역에 진입할 수 있다. 임계 영역을 빠져나오면 잠금을 되돌려 줘서 다른 쓰레드가 잠금을 얻을 수 있도록 한다.
뮤텍스 메커니즘의 특징을 정리했다.
Atomicity - mutex 잠금(lock)는 최소단위 연적(atomic operation)	으로 작동한다. 이말의 뜻은 하나의 쓰레드가 mutex 를 이용해서 잠금을 시도하는 도중에 다른 쓰레드가 mutex 잠금을 할수없도록 해준다는 뜻이다. 한번에 하나의 mutex 잠금을 하도록 보증해준다.
Singularity - 만약 스레드가 mutex 잠금을 했다면, 잠금을 한 쓰레드가 mutex 잠금을 해제 하기 전까지 다른 어떠한 쓰레드도 mutex 잠금을 할수 없도록 보증해준다.
Non-Busy Wait - 바쁜대기 상태에 놓이지 않는다는 뜻으로, 하나의 쓰레드가 mutex 잠금을 시도하는데 이미 다른 쓰레드가 mutex 잠금을 사용하고 있다면 이쓰레드는 다른 쓰레드가 락을 해제하기전까지 해당 지점에 머물러 있으며 이동안은 어떠한 CPU 자원도 소비하지 않는다 (이를테면 sleep).

## 뮤텍스 만들기
뮤텍스를 생성하기 위해서 우리는 먼저, 뮤텍스정보를 저장하기 위한 타입인 pthread_mutex_t 를 선언해주고 이것을 초기화 해주어야 한다. 선언과 초기화의 가장 간단한 방법은 PTHREAD_MUTEX_INITIALIZER 상수를 할당하는 것으로 아래와 같이 사용할수 있다.
**pthread_mutex_t a_mutex = PTHREAD_MUTEX_INITIALIZER;**
or
**int pthread_mutex_init(pthread_mutex_t * mutex, const pthread_mutex_attr *attr);**

## 뮤텍스 잠금, 잠금해제, 제거
뮤텍스 잠금을 위한 함수로는 pthread_mutex_lock() 함수를 제공한다. 이 함수는 해당 뮤텍스에 대해서 잠금을 시도하는데, 만약 잠그려는 뮤텍스가 다른 쓰레드에 의해서 이미 잠겨있다면, 잠금을 얻을수 있을때까지 - 이미 잠근 다른 쓰레드가 뮤텍스의 잠금을 해제할때까지 - **봉쇄(블럭)**되게 된다. 다음은 이러한 뮤텍스 잠금을 얻기 위한 지원함수들이다.

- int pthread_mutex_lock(pthread_mutex_t *mutex);
- int pthread_mutex_trylock(pthread_mutex_t *mutex);
- int pthread_mutex_destory(pthread_mutex_t *mutex);

pthread_mutex_trylock() 를 사용하면 잠금을 얻을수 없을경우 해당 코드에서 블럭되지 않고 바로 에러코드를 돌려준다. 즉 pthread_mutex_lock 의 비봉쇄 버젼이라고 생각하면 된다.
뮤텍스 잠금을 얻은후 해당 영역에서의 작업을 마친후 잠금을 해제하기 위해서 사용한다. 
- int pthread_mutex_unlock(pthread_mutex_t *mutex);

더이상 뮤텍스를 사용할일이 없다면 pthread_mutex_destory 를 이용해서 뮤텍스 자원을 제거(free) 하도록 한다. 만일 뮤텍스자원을 사용하는 **쓰레드가 하나라도 존재한다면 에러코드(EBUSY)를 리턴**한다. 그러므로 모든 쓰레드의 뮤텍스에 대해서 pthread_mutex_unlock 을 이용해서 잠겨져야만 뮤텍스 제거가 성공할수 있다. **성공할경우 0**을 넘겨준다.

## 작동 프로세스
작동 프로세스는 어떻게 mutex 잠금과 조건변수를 이용해서 임계영역을 보호하고 구조체의 값의 변경시점을 알수 있는지에 대한 내용을 중심으로 해서 기술할것이다.

~~~
  thread 2  
  while (1) 
  {
      mutex 잠금을 얻는다. 
      // 임계영역 시작 ----------------------------------------------
      구조체에 접근해서 값을 가져온다.  
      구조체 멤버변수의 값을 변경한다.(2씩 더한다)
      pthrad_cond_signal 를 이용해서 조건변수를 통해 신호를 보낸다. 
      // 임계영역 끝 ------------------------------------------------
      mutex 잠금을 돌려준다.
	  sleep(1);
  } 

  thread 3
  while(1)
  {
      mutex 잠금을 얻는다. 
      // 임계영역 시작 ----------------------------------------------
      pthread_cond_wait 를 이용해서 조건변수를 통해 신호가 오는지 기다린다.   
      if (신호가 도착한다면)
          두개의 구조체 멤버변수의 값을 덧셈 하고 이를 출력한다. 
      // 임계영역 끝 ------------------------------------------------
      mutex 잠금을 돌려준다.
  }
~~~

~~~
#include <pthread.h>
#include <string.h>
#include <unistd.h>

pthread_mutex_t mutex_lock   = PTHREAD_MUTEX_INITIALIZER;
pthread_cond_t   thread_cond  = PTHREAD_COND_INITIALIZER;

struct com_data
{
    int a;
    int b;
};

struct com_data mydata;

void *do_write(void *data)
{
    mydata.a = 0;
    mydata.b = 0;
    while(1)
    {
        pthread_mutex_lock(&mutex_lock);
        mydata.a = random() % 6000;
        mydata.b = random() % 6000;
        pthread_cond_signal(&thread_cond);
        pthread_mutex_unlock(&mutex_lock);
        sleep(2);
    }
}

void *do_read(void *data)
{
    while(1)
    {
        pthread_mutex_lock(&mutex_lock);
        pthread_cond_wait(&thread_cond, &mutex_lock);
        printf("%4d + %4d = %4d\n", mydata.a, mydata.b, mydata.a + mydata.b); 
        pthread_mutex_unlock(&mutex_lock);
    }
}

int main()
{
    pthread_t p_thread[2];
    int thr_id;
    int status;
    int a = 1;
    int b = 2;

    thr_id = pthread_create(&p_thread[0], NULL, do_write, (void *)&a); 
    thr_id = pthread_create(&p_thread[1], NULL, do_read, (void *)&b);

    pthread_join(p_thread[0], (void **) status);
    pthread_join(p_thread[1], (void **) status);

    return 0;
}
~~~

조건변수에는 pthread_cond_signal(3) 과 ptherad_cond_wait(3) 를 이용해서 신호를 주고, 기다리는 방식을 사용한다고 했다. 그렇다면 생각할수 있는게, 과연 신호가 실시간으로 전달이 될것이란걸 믿을수 있을까?

실시간으로 전달되는지 아닌지가 중요한 이유는 쓰레드가 신호를 보내고 나서 신호를 잘받았는지 기다리지 않고 바로 다음으로 넘어가 버리기 때문이다.

이건 꽤 중요한 문제가 될수도 있다. 왜냐하면 만약 신호가 실시간으로 전달되지 않는다면 신호가 미쳐 전달되기 전에 어떤 데이타가 변경되어 버리는 경우가 발생할수 있기 때문이다.
~~~
            쓰레드 공유변수 A = 0

 thread 1                                    thread 2
 while(1)                                    while(1)
 {                                           {
     쓰레드 공유변수 A++      
     신호 보냄           ------------------>     신호  기다림
  }                                           } 
~~~                                           
			
위의 상황을 생각해 보자 최초 공유변수 A 에 0 이 들어간다. thread 1 에서 여기에 1 을 증가시키고 신호를 보낸다. thread 2 는 신호를 받고 A 의 값을 읽어 들여서 이것을 100 으로 나눈다. 그런데 신호가 늦게 보내져서 - thread 1 의 loop 회전속도가 신호를 보내는 시간보다 빠른경우 - thread 2 에서 신호를 미쳐 받기전에 A ++ 이 한번더 실행되고 A 의 값은 2가 될것이다. 이때 서야 thread 2 로 신호가 전달되었다면 결국 thread 1 에서는 2번의 데이타를 보냈는데 thread 1 는 한번의 연산만 실행한것으로 데이타 하나를 잃어 버린것과 같은 문제가 발생해 버린다.
신호는 매우 빠른 시간에 전달됨으로 보통의 경우 신호전달시간을 염두에 두어야 하는 경우는 발생하지 않을것이다. 하지만 불행하게도 염두에 두어야 하는 경우가 발생하기도 한다.

물론 우리 프로그래머들의 사전에 불가능이란 없으므로 위의 문제도 간단하게 해결가능 하다. 조건변수를 2개 쓰면 된다. thread 1 에서 신호를 보냈다면, thread 1 은 다음 루틴으로 넘어가기 전에 thread 2 에서 넘어오는 신호를 기다리도록 하면 될것이다. thread 2 는 thread 1의 신호를 받은뒤 thread 1으로 신호를 보내게 될것임으로 반드시 신호가 전달될것을 확신할수 있을것이다. 2개의 조건변수를 지원하기 위해서 2개의 mutex 잠금이 필요할것이다. 여기에서는 그 구현까지 설명하지는 않을것이다. 조금만 생각해보면 간단하게 구현 가능할것이기 때문이다.
~~~
 thread 1                                    thread 2
 while(1)                                    while(1)
 {                                           {
     쓰레드 공유변수 A++      
     신호 1 보냄           ----------------->     신호 1 기다림
	 신호 2 기다림         <----------------      신호 2 보냄
     ....                                            ....
 }                                           } 
~~~  			
신호의 전달에 걸리는 시간은 운영체제에 따라 상당한 차이를 보인다. 그러므로 이러한 오차시간까지도 염두에 두어야할 상황이 발생한다면 시간테스트를 해야할것이다.

# 병렬 프로그램

##  POSIX 와 Thread-safety
멀티 쓰레드 환경에서는 사용하려는 함수가 재진입가능한지를 검토해야 한다
asctime()함수는 프로세스의 메모리영역에 공간을 할당하고, 그에 대한 포인터를 돌려준다. 데이터 영역이 독립적이지 않기 때문에 다른 쓰레드에서 asctime함수를 호출하면, 프로세스 데이터가 변해 버리는 문제가 발생한다.

~~~
#include <stdio.h>
#include <unistd.h>
#include <stdlib.h>
#include <pthread.h>
#include <time.h>

#define MAX_THREAD_NUM  1
void *t_function(void *data)
{
    time_t current_time ;
    struct tm *mytm;
    char *time_str;

    current_time = time((time_t *)NULL);
    mytm = localtime(&current_time);

    time_str = asctime(mytm);
    printf("Child Thread Start Time : %s", time_str);
    sleep(5);
}

int main(int argc, char **argv)
{
    pthread_t p_thread[2];
    int thr_id;
    int status;
    int i = 0;
    time_t current_time ;
    struct tm *mytm;
    char *time_str;

    current_time = time((time_t *)NULL);
    mytm = localtime(&current_time);
    time_str = asctime(mytm);
    printf("Main Thread Start Time 1 : %s", time_str);
    sleep(10);

    for( i = 0; i < MAX_THREAD_NUM; i++)
    {
        thr_id = pthread_create(&p_thread[i], NULL, t_function, (void *)&i);
        if (thr_id < 0)
        {
            perror("thread create error : ");
            exit(0);
        }
    }
    pthread_join(p_thread[0], NULL);
    printf("Main Thread Start Time 2 : %s", time_str);
    return 0;
}
~~~

멀티 쓰레드 프로그램에서, 재진입을 보장하지 않는 함수를 사용했을 때 발생하는 문제를 보여주고 있다.
t_function 쓰레드함수에서 asctime을 호출하면서 메모리 할당된 영역의 데이터를 변경해 버렸기 때문이다. 다음은 재진입하지 않는 함수들이다.
- POSIX.1 프로세스 환경 관련 함수들 : getlogin(), ttyname()
- C언어 함수들 : asctime(), gmtime(), localtime()
- POSIX.1 시스템 데이터 베이스 함수 getgrgid(), getgrnam(), getpwuid(), getpwnam()
이 문제를 해결 하기 위해서 재진입 가능한 별도의 함수를 사용해야 한다. 함수에 **_r**이 붙은 함수는 재진입 가능한 버전의 함수임을 의미한다. **asctime**함수의 재진입 가능한 함수는 **asctime_r**함수다. 당연히 멀티 쓰레드 프로그래밍에서는 _r이 붙은 이름의 함수를 사용해야 한다.

errno
-----
errno는 external 전역 변수다. 그러므로 멀티 쓰레드 환경에서 에러를 검사하기 위한 목적으로 사용하기 힘들다. 최근에 호출한 함수의 errno값인지를 장담할 수 없기 때문이다.

- 재진입 가능한 버전의 함수 즉, _r이 붙은 함수를 사용한다.
- 뮤텍스()등으로 전역 데이터에 대한 접근을 제어한다.
- gcc의 경우 _REENTRANT 정의해서 재진입을 보장할 수 있다. 재진입 가능한 함수와 그렇지 않은 함수가 존재 할때, 재진입 가능한 함수를 링크한다. 또한 errno와 같은 external 전역 변수를 각 쓰레드 마다 사용할 수 있도록 해준다.

## 재진입 가능한 사용자 정의 함수 만들기
- 전역변수를 사용하지 않는다.
- 전역변수의 포인터를 반환하지 않는다. 대신 인자로 데이터를 넘겨받도록 한다. POSIX의 _r계열 함수가 이런 방식을 사용한다.
- 공유되는 자원은 접근제어를 한다.
- 재진입하지 않는 함수는 호출하지 않는다.
- 데이터 공간을 함수가 아닌 호출자가 제공하도록 한다.

# 병렬 프로그래밍에서 주의 해야할 문제들

## ABA 문제
A 쓰레드와 B 쓰레드가 있다.
- A 쓰레드가 메모리의 값을 읽는다. 이런 저런 연산을 한다.
- B 쓰레드가 메모리의 값을 읽는다. 이런 저런 연산을 한다.
- A 쓰레드가 메모리에 연산 값을 쓴다.
- B 쓰레드가 메모리에 연산 값을 쓴다.
이를 ABA 문제라고 한다. 정상적으로 연산이 되려면, 다음과 같은 흐름을 가져야 한다.
- A 쓰레드가 메모리의 값을 읽는다. 이런 저런 연산을 한다.
- A 쓰레드가 연산 값을 메모리에 쓴다.
- B 쓰레드가 메모리의 값을 읽는다. 이런 저런 연산을 한다.
- B 쓰레드가 메모리에 여산 값을 쓴다.
이 문제는 세마포어()나 뮤텍스()등으로 임계영역에 대한 접근을 제어하는 것으로 해결할 수 있다.

## killer-tolerance
잠금을 얻은 쓰레드가 어떤 이유로 잠금을 풀지 않고 종료되더라도 다른 쓰레드에 영향을 미치면 안된다. 즉 스레드가 죽으면 자동적으로 잠금을 돌려줘야 한다. 만약 pthread()우ㅢ mutex()를 이용해서 잠금을 생성했다면, killer-tolerance를 걱정할 필요가 없다. 알아서 해제 시켜주기 때문이다. 다음의 코드를 테스트 해보자.

~~~
#include <stdio.h>  
#include <unistd.h>  
#include <pthread.h>  
 
int ncount;    // 쓰레드간 공유되는 자원 
pthread_mutex_t  mutex = PTHREAD_MUTEX_INITIALIZER; // 쓰레드 초기화 
 
void* do_loop(void *data) 
{ 
    int i; 
    for (i = 0; i < 10; i++) 
    { 
        pthread_mutex_lock(&mutex); // 잠금을 생성한다. 
        printf("loop1 : %d\n", ncount); 
        ncount ++; 
        if(i == 10) return;           // 잠금을 풀지 않고 스레드를 종료한다.
        pthread_mutex_unlock(&mutex); // 잠금을 해제한다. 
        sleep(1); 
    } 
} 
 
void* do_loop2(void *data) 
{ 
    int i; 
 
    // 잠금을 얻으려고 하지만 do_loop 에서 이미 잠금을  
    // 얻었음으로 잠금이 해제될때까지 기다린다.   
    for (i = 0; i < 10; i++) 
    { 
        pthread_mutex_lock(&mutex); // 잠금을 생성한다. 
        printf("loop2 : %d\n", ncount); 
        ncount ++; 
        pthread_mutex_unlock(&mutex); // 잠금을 해제한다. 
        sleep(2); 
    } 
}     
 
int main() 
{ 
    int       thr_id; 
    pthread_t p_thread[2]; 
    int status; 
    int a = 1; 
 `
    ncount = 0; 
    thr_id = pthread_create(&p_thread[0], NULL, do_loop, (void *)&a); 
    sleep(1); 
    thr_id = pthread_create(&p_thread[1], NULL, do_loop2, (void *)&a); 
 
    pthread_join(p_thread[0], (void *) &status); 
    pthread_join(p_thread[1], (void *) &status); 
 
    status = pthread_mutex_destroy(&mutex); 
    printf("code  =  %d", status); 
    printf("programing is end"); 
    return 0; 
} 
~~~
세마포어를 사용할 때, 혹은 파일 잠금이나 기타 IPC()를 사용할 때 쓰레드의 종료가 잠금에 미치는 영향에 대해서 충분히 고려해야 한다.

### async-signal-safety
특정 함수 A가 잠금을 얻은 상태에서, 시그널이 발생해서 시그널 핸들러가 실행됐다. 그런데, 시그널 핸들러가 함수 A를 다시 호출한다면, 락을 얻은 상태에서 다시 락을 얻을려고 실행하기 때문에 영원히 봉쇄될 것이다. 이런 경우가 자주 발생할 것 같지는 않다. 그냥 상식적으로 시그널 핸들러가 자신을 호출한 함수를 호출하지 않도록 작성하면 이 문제를 회피할 수 있을 것이다.

### preemption-tolerance
잠금을 얻은 스레드가 휴면상태에 있을 때, 다른 스레드에 영향을 미치면 안된다. 다분히 논리적인 문제라고 할 수 있다. 즉 스레드는 마땅히 휴면상태에 있어야 할때, 휴면 상태에 놓여야 한다. 하지만 잠금을 얻은 상태에서 오랜 시간 임계영역에 머무르는 것은 바람직하지 않을 것이다.

### processor heap
메모리를 할당 할때, 프로세스의 heap영역을 사용하는 것은 지양하자. 프로세스의 heap을 사용할 경우 false sharing 문제가 발생할 수 있다. 또한 자원 공유를 위한 락에는 많은 비용이 들어간다.

### false sharing
멀티 코어 CPU에서 발생할 수 있는 문제다. 멀티 코어 CPU에서는 데이터를 word 단위로 읽어오는 대신 메모리 I/O 효율성을 위해서 cache-line로 읽어오는데, 이대 문제가 생길 수 있다. 두개의 메모리 연산이 동일한 캐쉬라인에서 실행될 경우, CPU<->Memory 버스 사이에서 하드웨어적인 락이 걸리는데, 이때 하드웨어적인 병목현상이 발생한다.

다음과 같은 방법으로 해결할 수 있다.
- OS가 관리해주는 메모리 단위인 페이지에서 같이 쓰는 데이터가 같이 올라가도록 할 것.
- 서로 다른 코어에서 접근할 수 있는 영역이면 캐쉬라인 단위로 떨어질 수 있게 적절히 패딩할 것
- 논리적으로 같이 사용되는 데이터라도, 성능을 위해서 서로 떨어트리는 것을 고려할 것.

아래는 윈도우 소스 같은데 
~~~
#include <windows.H>
#include <stdio.H>
#include <tchar.H>
 
volatile int data1;
volatile int data2;
 
DWORD CALLBACK TestThread1(void* /*arg*/)
{
    for (int i = 0; i < 500000000; ++i)
        data1 = data1 + 1;
    return data1;
}
 
DWORD CALLBACK TestThread2(void* /*arg*/)
{
    for (int i = 0; i < 500000000; ++i)
        data2 = data2 + 1;
    return data2;
}
 
int _tmain(int argc, _TCHAR* argv[])
{
    HANDLE thread[2];
    SetPriorityClass(GetCurrentProcess(), HIGH_PRIORITY_CLASS);
 
    DWORD startTime = GetTickCount();
    thread[0] = CreateThread(NULL, 0, TestThread1, (LPVOID)0, 0, NULL);
    thread[1] = CreateThread(NULL, 0, TestThread2, (LPVOID)0, 0, NULL);
    WaitForMultipleObjects(2, thread, TRUE, INFINITE);
    _tprintf(_T("%d\n"), GetTickCount() - startTime);
    return 0;
}
~~~

# Thread Pooling

## Thread Pooling 이란
pool 의 사전적인 뜻을 찾아보면 연못, 저수지, 수영장 풀 등 "무엇을 담아놓는" 의 뜻을 가진다. 이대로 해석하자면 Thread Pooling 이란 쓰레드를 담아 놓는 용기(메모리가 될것이다) 를 뜻하며, 프로그래밍 측면에서 해석하자면, **"미리 쓰레드를 할당시켜 놓는기법"** 을 뜻한다.
**Thread Pooling 은 이러한 반복적인 쓰레드의 생성/소멸에 의한 비효율적인 측면을 없애고자 하는 목적으로 만들어진 프로그래밍 기법이다.**

## Thread Pool의 구현방식
개념적으로 보자면 Thread Pool 을 구성하는건 매우 간단하다. 생성하고자 하는 크기만큼 ptread_create() 함수를 돌리면 되기 때문이다.

하지만 이건 어디까지나 개념적인 것으로 대부분의 경우 각각의 쓰레드를 스케쥴링 해주어야 함으로, 때에 따라서는 구현을 위해서 매우 복잡한 프로그래밍 기법을 동원해야 할때도 있다. 간단히 웹 서버를 Thread Pool 로 구현한다고 가정을 해보자 - 보통 웹서버는 HTTP 의 특성상 연결과/종료가 빈번하게 일어 남으로 쓰레드풀을 사용할경우 많은 이익을 얻을수 있다 -, 만약 100 개의 Thread 를 미리 생성시켰고, 각각의 Thread 는 하나의 클라이언트 연결을 처리한다고 가정했을때, main 쓰레드는 accept(2) 를 통해서 클라이언트를 받아들였을때, accept() 로 만들어진 소켓 지정번호를 미리 만들어진 100 개의 쓰레드중 "놀고" 있는 쓰레드에게 넘겨주어야 할것이다. 그러기 위해서는 **main 쓰레드에서 각각의 쓰레드 상태를 유지해서 적당한 쓰레드에게 파일지정자**를 넘겨줘야 할것이다.

그나마 위의 경우는 하나의 쓰레드가 하나의 연결을 처리함으로 어렵지 않게 구현하겠지만, 만약 100개의 쓰레드가 있고, 거기에 각각의 쓰레드가 10개 씩의 클라이언트 연결을 처리하도록 구성한다면, 거기에다가 적당한 로드밸런싱 기능 까지 포함시키고자 한다면, 구현이 꽤 복잡해 질수도 있다.
![Alt text](./thread/thread_3.png)

**Thread Pool 에 들어있는 각각의 쓰레드를 관리하기 위해서는 필수적으로 각각의 쓰레드의 상태를 가지고 있는 Schedul 자료구조 를 가지고 있어야한다. 그래야만 MAIN THREAD 에서 쓰레드 상태를 확인해서 적당한 쓰레드로 작업분배가 가능할것이기 때문이다.** - 실제 Linux 커널도 각각의 task 의 스케쥴링을 위해서 task 구조체를 유지한다. 

## 구현 프로세스
~~~
스케쥴관련 자료구조
{
   현재 연결된 클라이언트수
   현재 처리해야될 클라이언트 소켓지시자

   쓰레드풀에 만들어진 쓰레드 상태 : 쓰레드풀 크기만큼의 배열 
   {
       0 이면 휴식상태 
       1 이면 작업상태 
       처리중인 소켓지시자
   }
};

main 함수시작
{
    아규먼트로 몇개의 쓰레드를 생성할지를 받음
    while(쓰레드 생성수만큼)
    {
        pthread_create 를 이용해서 쓰레드 생성
        // 통신쓰레드 함수
        {
            WAIT:
            main 쓰레드가 깨우길 기다린다. 
            만약 main 쓰레드로 부터 깨움이 있다면   
            {
                스케쥴 자료구조->현재 처리해야될 소켓지시자 를 읽어온다. 
                스케쥴 자료구조->자신의 상태를 1로 세팅한다.  
                스케쥴 자료구조->처리중인 소켓지시자를 세팅한다. 
                while(1)
                {
                    클라이언트와 통신한다. 
                    만약 에러가 발생하면 
                    {
                        스케쥴 자료구조->처리중인 소켓지시자를 0으로 세팅  
                        스케쥴 자료구조->자신의 상태를 0으로 세팅
                        스케쥴 자료구조->현재 연결된 클라이언트수 --; 
                        goto WAIT:
                    }
                }
            }
        } 
    }

    // main 쓰레드
    while(1)
    {
        만약 accept 를 통해서 연결이 발생한다면
        {
            스케쥴관련 자료구조->현재연결된 클라이언트수가 MAX 를 초과하지 않았다면
            {
                스케쥴관련 자료구조->현재연결된 클라이언트수 ++; 
                스케쥴관련 자료구조->현재처리해야될 클라이언트 소켓지시자 = accept();
                스케줄관련 자료구조->쓰레드풀에 만들어진 쓰레드상태 가 0인 
                  쓰레드를 찾아서 해당 쓰레드를 깨운다.  
            }
            그렇지 않고 초과했을경우
            {
                클랑리언트에게 에러메시지를 전송한다. 
            }
        }
    }
}
~~~

구현은 구현하는 프로그래머가 상황에 따라서 선택하기 나름이긴 하지만 보통은 위의 방법을 기본으로 해서, 약간의 변경을 가하는 정도가 될것이다. 위의 슈도코드를 보면 main 쓰레드에서 accept 를 받으면 휴식상태에 있는 쓰레드를 깨운다고 되어있는데, 이때 깨우기 위해서는 쓰레드 조건변수를 사용하면 될것이다.
그렇다면 **스케쥴관련 자료구조**는 어떻게 구현하는게 쉬운방법인지 생각해보도록 하자. 구현하는 방법은 프로그래머 맘이겠지만, 구현하고자 한다면 **multimap 을 이용해서 구현**할것이다. 

~~~
// 쓰레드 정보 구조체
struct ph
{
   int sockfd;    // 처리중인 소켓지정번호
   int index_num; // 쓰레드의 인덱스 번호
};

// 쓰레드 구조체 MAP
multimap<int, struct ph> phinfo;

struct schedul_info
{
    int client_num;      // 총 연결중인 클라이언트수 
    int current_sockfd;  // 가장최근에 연결된 소켓지정번호
    phinfo mphinfo;      // 쓰레드 구조체 map
} 
~~~

멀티맵의 key 는 쓰레드의 **활성화 여부로 1 혹은 0이** 된다. 그리고 value 는 해당 쓰레드 정보가 될것이다. 이렇게 멀티맵으로 만든이유는 간단하다. **멀티맵은 정렬연관 컨테이너 임으로 key 를 기준으로 자동적으로 정렬**이 될것이다. 만약 **첫번째 쓰레드가 처리중(1)로 변경되었다면 이 원소는 multimap 의 가장 뒤로 정렬**이 될것이다. 그럼으로 우리는 클라이언트의 수가 총연결가능한 클라이언트수(Thread Pool 에 생성된 쓰레드수)	를 초과하지 않는한 phinfo.begin() 으로 가져온 **쓰레드는 휴식상태(0) 이라는걸 믿을수 있게** 된다. 다시 말해서 복잡해서 쓰레드상태가 0인지 1인지 처음부터 검사할 필요가 없다는 뜻이다.

![Alt text](./thread/thread_4.png)

**사실 multimap 을 쓴다면 굳이 "현재 연결된 클라이언트 수" 를 유지하기 위해서 별도의 변수를 둘 필요가 없을것이다. multimap 에서 제공하는 count() 를 이용해서 key 가 "1" 인 요소의 수를 구하면 되기 때문이다. 만약 multimp 의 begin() 값이 1 이라면 MAX 클라이언트가 가득찼다는걸 의미할것이다.**
물론 multimap 의 경우 기본적으로 key 값의 수정은 허용하지 않기 때문에 0 을 1로 변경할경우 실제로는 0 을 가지는 요소를 삭제하고, 1을 가지는 새로운 요소를 삽입하는 방식을 취해야 할것이다. 마찬가지로 클라이언트가 종료해서 1을 0으로 변경할때에도 삭제/인서트를 해야할것이다. Value(값) 는 그대로 복사해서 삭제/인서트를 해야 한다.

~~~
#include <map>
#include <iostream>
#include <pthread.h>
#include <unistd.h>
#include <stdlib.h>
#include <time.h>
#include <sys/socket.h>
#include <sys/stat.h>
#include <signal.h>
#include <arpa/inet.h>
#include <sys/un.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

// 최대 쓰레드 POOL 크기
#define MAX_THREAD_POOL 256
using namespace std;

// 전역 쓰레드 구조체 
typedef struct _ph
{
    int sockfd;    // 현재 사용중인 소켓 fd
    int index_num; // 인덱스 번호
} ph;

// 전역쓰레드 구조체로써 
// 현재 쓰레드 상황을 파악함
struct schedul_info
{
    int client_num;       // 현재 연결된 클라이언트수
    int current_sockfd;   // 가장최근에 만들어진 소켓지시자
    multimap<int, ph> phinfo; 
};

// 각 쓰레드별 조건변수
pthread_cond_t *mycond;
// 쓰레드 동기화를 위한 조건변수
pthread_cond_t async_cond = PTHREAD_COND_INITIALIZER;

// 각 쓰레드별 조건변수의 크리티컬세션 지정을 위한 
// 뮤텍스 
pthread_mutex_t mutex_lock= PTHREAD_MUTEX_INITIALIZER; 
// 쓰레드 동기화용 조건변수의 크리티컬세션 지정을 위한 
// 뮤텍스
pthread_mutex_t async_mutex = PTHREAD_MUTEX_INITIALIZER;

// 클라이언트와의 통신용 쓰레드
void *thread_func(void *data);
// 현재 클라이언트 상태 모니터용 쓰레드
// 한마디로 디버깅용 
void *mon_thread(void *data);

schedul_info s_info;

// 메인 함수
int main(int argc, char **argv)
{
    int i;
    ph myph;
    int status;
    int pool_size = atoi(argv[2]);
    pthread_t p_thread;
    struct sockaddr_in clientaddr, serveraddr;
    int server_sockfd;
    int client_sockfd;
    int client_len;    

    // 풀사이즈 검사
    if ((pool_size < 0) || (pool_size > MAX_THREAD_POOL))
    {    
        cout << "Pool size Error" << endl;
        exit(0);    
    }

    // Make Socket
    if ((server_sockfd = socket(AF_INET, SOCK_STREAM, 0)) < 0)
    {
        perror("error : ");
        exit(0);
    }

    // Bind
    bzero(&serveraddr, sizeof(serveraddr));
    serveraddr.sin_family = AF_INET;
    serveraddr.sin_addr.s_addr = htonl(INADDR_ANY);
    serveraddr.sin_port = htons(atoi(argv[1]));

    if (bind (server_sockfd, (struct sockaddr *)&serveraddr, 
                        sizeof(serveraddr)) == -1) 
    {
        perror("bind error : ");
        exit(0);
    }

    // Listen
    if (listen(server_sockfd, 5) == -1)
    {
        perror("listen error : ");
        exit(0);
    }

    // 쓰레드 갯수만큼 조건변수 생성 
    mycond     = (pthread_cond_t *)malloc(sizeof(pthread_cond_t)*pool_size);

    // 쓰레드 전역변수 초기화
    s_info.client_num = 0;
    s_info.current_sockfd = 0; 

    // 쓰레드 POOL 새성
    for (i = 0; i < pool_size; i++)
    {
        memset((void *)&myph, 0x00, sizeof(myph));
        myph.index_num = i;
        s_info.phinfo.insert(pair<int, ph>(0, myph));

        // 조건변수를 이용해서 쓰레드간 동기화를 실시한다.
        pthread_mutex_lock(&async_mutex);
        if (pthread_create(&p_thread, NULL, thread_func, (void *)&i) < 0)
        {
            perror("thread Create error : ");
            exit(0);    
        }    
        pthread_cond_wait(&async_cond, &async_mutex); 
        pthread_mutex_unlock(&async_mutex);
    }

    // 디버깅용 쓰레드 생성
    pthread_create(&p_thread, NULL, mon_thread, (void *)NULL);

    // MAIN THREAD accept wait
    client_len = sizeof(clientaddr);

    // 클라이언트 ACCEPT 처리를 위한 
    // MAIN 쓰레드 
    while(1)
    {
        multimap<int, ph>::iterator mi;
        client_sockfd = accept(server_sockfd, (struct sockaddr *)&clientaddr, 
                                        (socklen_t *)&client_len);
        if (client_sockfd > 0)
        {
            // 만약 쓰레드풀이 가득찼다면 클라이언트 연결을
            // 종료시킨다.
            mi = s_info.phinfo.begin();
            if (mi->first == 1)
            {
                //error message send to client_sockfd
                cout << "SOCKET IS FULL" << endl;    
                close(client_sockfd);
            }
            // 그렇지 않다면 연결을 받아들이고 
            // 클라이언트 전역 변수를 세팅한다. 
            // 세팅후 해당 처리쓰레드에게 시그널을 보내어서 
            // 처리하게 한다. 
            else
            {
                ph tmpph;
                int psockfd;
                int pindex_num;
                s_info.current_sockfd = client_sockfd;

                tmpph.sockfd = client_sockfd;
                tmpph.index_num = mi->second.index_num;
                s_info.phinfo.erase(mi);    
                s_info.phinfo.insert(pair<int, ph>(1,tmpph));
                s_info.client_num ++;
                cout << "SEND SIGNAL " << mi->second.index_num << endl;     
                pthread_cond_signal(&mycond[mi->second.index_num]);    
            }
        }
        else
        {
            cout << "ACCEPT ERROR " << endl;    
        }
    }
    pthread_join(p_thread, (void **)status);
}

void *thread_func(void *data)
{
    char buf[255];
    int mysocket;
    int mynum = *((int *)data); 
    multimap<int, ph>::iterator mi;
    // 쓰레드 동기화용 조건변수
    pthread_mutex_lock(&async_mutex);
    pthread_cond_signal(&async_cond);
    pthread_mutex_unlock(&async_mutex);

    cout << "Thread create " << mynum << endl;
    while(1)
    {
        // MAIN 쓰레드로 부터 신호를 기다린다. 
        // 신호가 도착하면 쓰레드 전역변수로 부터 
        // 현재 처리해야할 소켓지정값을 가져온다. 
        pthread_mutex_lock(&mutex_lock);
        pthread_cond_wait(&mycond[mynum], &mutex_lock);
        mysocket = s_info.current_sockfd;
        pthread_mutex_unlock(&mutex_lock);
        memset(buf, 0x00, 255);    

        // 데이타를 처리한다. 
        // 만약 quit 문자열을 만나면 
        // 쓰레드 전역변수를 세팅한다음 연결종료 한다. 
        while(1)
        {
            read(mysocket, buf, 255);
            if (strstr(buf, "quit") == NULL)
            {
                write(mysocket, buf, 255);
            }
            else
            {
                mi = s_info.phinfo.begin();
                while(mi != s_info.phinfo.end())
                {
                    cout << "search " << mi->second.index_num << endl;
                    if (mi->second.index_num == mynum)
                    {
                        ph tmpph;
                        tmpph.index_num = mynum;
                        tmpph.sockfd = 0;
                        s_info.phinfo.erase(mi);
                        s_info.phinfo.insert(pair<int, ph>(0, tmpph));
                        s_info.client_num --;
                        close(mysocket);
                        break;
                    }
                    mi ++;
                }
                break;
            }
            memset(buf, 0x00, 255);    
        }
    }
}

void *mon_thread(void *data)
{
    cout << "moniter thread" << endl;
    while(1)
    {
        sleep(10);
        multimap<int, ph>::iterator mi;
        mi = s_info.phinfo.begin();
        cout << "size " << s_info.phinfo.size() << endl;
        while(mi != s_info.phinfo.end())
        {
            cout << mi->first << " : " << mi->second.index_num 
                 << " : " << mi->second.sockfd << endl; 
            mi ++;
        }
    }
}
~~~
이 프로그램은 2개의 인자를 받아들이며, 클라이언트의 입력을 되돌려주는 일을한다 (echo 서버). 첫번째 인자는 서비스할 PORT 번호이고, 두번째 인자는 쓰레드 생성갯수이다. 프로그램은 인자의 정보를 이용해서 PORT 를 열고 클라이언트를 받아들인다. 클라이언트가 연결하면, Thread Pool 에 남는 공간이 있는지를 확인하고, 남는 공간이 있다면 클라이언트와 통신하게 된다.

## 결론
쓰레드 풀은 보통 매우 효율적인 성능을 보장해주는 어플리케이션의 작성을 위해서 사용되어짐으로, 가능한한 빠른 쓰레드간 전환이 가능하도록 고민해서 코딩을 해야 한다. 위의 경우 쓰레드간 전환을 위해서 multimap 을 사용하고 있는데, accept 가 들어왔을경우 해당 클라이언트에 대한 쓰레드 할당은 매우 빠르다고 볼수 있을것이다. 그러나 종료할경우에는 multimap 의 첫번째 원소부터 마지막번 원소까지 search 해야 한다. 이것은 매우 비효율적임으로 개선할 여지가 있다. **가장 간단하게 생각할수 있는 것은 multimap 의 key 값이 1인 원소내에서만 검색하는 것이다. 우리는 쓰레드 풀의 크기와 현재 연결된 클라이언트의 수를 알고 있음으로, multimap 의 몇번째 요소부터 key 값이 1인지를 계산해 낼수 있기 때문**이다. 이렇게 할경우 약간의 **시간단축효과를 기대**할수 있을것이다.

![Alt text](./thread/thread_5.png)

이 **시간단축효과는 연결된 클라이언트의 수가 전체 POOL 사이즈에 비례해서 작을 수록 커질**것이다.

# 쓰레드 객체의 자원 정리

쓰레드를 클래스를 이용해서 객체화 할경우, 쓰레드를 detach 상태로 실행시키는 경우가 많다. 이 경우 pthread_joinc()함수를 이용하지 않게 됨으로, 부모 쓰레드는 자식쓰레드의 상태를 정확하게 알기 힘들고, 때문에 자원정리에 문제가 발생할 수 있다. 자식 쓰레드가 자신의 종료시점을 명확히 알고 있다면, 소멸자를 이용해서 필요한 해제를 할 수 있겠지만, pthread_cancel()등 비동기적인 방법으로 종료 시킬경우 문제가 될 수 있다.
여러가지 방법이 있겠으나, **pthread_cleanup_push()와 pthread_cleanpu_pop() 함수를 이용해서 이 문제를 해결**했다.

## 테스트 서버 프로그램
간단한 테스트 서버 프로그램을 만들어 보도록 하겠다. 이 프로그램은 echo 클라이언트의 thread() 버젼이다. 연결이 들어오면 부모쓰레드는 새로운 자식쓰레드를 생성한다. 또한 부모쓰레드는 select()를 이용하여 표준입력을 기다린다. 이 표준입력을 통해서 특정 자식쓰레드 종료나 자식쓰레드 상태 정보제공등과 같은 일을 하게 된다.

![Alt text](./thread/thread_6.png)
~~~
#include <pthread.h>
#include <stdlib.h>
#include <unistd.h>
#include <iostream>
#include <signal.h>

#include <sys/stat.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <arpa/inet.h>

#include <string.h>
#include <map>

using namespace std;

#define MAXLEN  256

map<int, pthread_t> Cinfo;
class tClass {
	private :
		pthread_t pt;
		int sockfd;
		char *buf;

		static void* callThread(void* arg);
		// 쓰레드 종료함수
		// 자신을 delete 한다.
		static void clean_up(void *arg)
		{
			delete (tClass *)arg;
		}

	public :
		// Client와 통신을 담당하는 함수로 쓰레드 함수인 callThread에서 
		// 호출한다.
		void tFunc(void) 
		{
			int readn;
			buf = (char *)malloc(MAXLEN);

			// 쓰레드 종료함수인 clean_up을 호출한다.
			// 인자로 쓰레드 자신의 포인터를 넘긴다.
			pthread_cleanup_push(clean_up, (void *)this);
			while (1) 
			{
				memset(buf, 0x00, MAXLEN);
				readn = read(sockfd, buf, MAXLEN);
				if(readn <=0)
				{
					perror("Socket Read Error:");	
					break;
				}
				write(sockfd, buf, strlen(buf));
			}
			pthread_exit((void *)NULL);
			// 쓰레드 종료함수를 pop시킨다.
			pthread_cleanup_pop(0);
		}

		tClass(int sfd) 
		{
			sockfd = sfd;
			buf = NULL;
			pt = 0;
			pthread_attr_t attr;

			// 쓰레드를 Detached 상태로 만든다.
			pthread_attr_init(&attr);
			pthread_attr_setdetachstate(&attr, PTHREAD_CREATE_DETACHED);
			pthread_create(&pt, &attr, callThread, this);
		}

		// 만들어진 쓰레드객체의 pthread_id를 리턴한다. 
		pthread_t tClassId()
		{
			return pt;
		}

		// 소멸자로
		// 쓰레드 자원과 전역 자료구조를 정리한다.
		~tClass(void) 
		{
			map<int, pthread_t>::iterator mi;
			if(buf) 
				free(buf);	
			if (sockfd)
			{
				mi = Cinfo.find(sockfd);
				if (mi!=Cinfo.end())
				{
					Cinfo.erase(mi);
			}
			close(sockfd);
		}
		printf("Thread Close\n");
	}
};


void* tClass :: callThread(void* arg) 
{
   tClass* tp;
   tp = (tClass*) arg;
   tp->tFunc();   
}

int main (int argc, char **argv) 
{
	pthread_t pthread;
	struct sockaddr_in clientaddr, serveraddr, myaddr;
	int maxfd = 0;
	fd_set fd_w;
	struct timeval timeout;

	class tClass *ClientClass;

	int server_sockfd, client_sockfd, client_len;

	if (argc != 2)
	{
		printf("Usage : %s [PORT]\n", argv[0]); 
		return 1;
	}

	if ((server_sockfd = socket(AF_INET, SOCK_STREAM, 0)) < 0)
	{
		perror("socket error:");
		return 1;
	}

	bzero(&serveraddr, sizeof(serveraddr));
	serveraddr.sin_family = AF_INET;
	serveraddr.sin_addr.s_addr = htonl(INADDR_ANY);
	serveraddr.sin_port = htons(atoi(argv[1]));

	if (bind(server_sockfd, (struct sockaddr *)&serveraddr,
					sizeof(serveraddr)) == -1)
	{
		perror("bind error:");
		return 1;
	}
	if (listen(server_sockfd, 5) == -1)
	{
		perror("listen error :");
		return 1;
	}
	client_len = sizeof(clientaddr);

	char buf[MAXLEN];

	// Main Thread
	// Accept 입력인 경우 새로운 쓰레드를 생성시키고
	// 표준입력인 경우 입력된 명령을 확인해서   
	// show일경우 쓰레드 정보를 보여주고
	// kill일 경우 해당 쓰레드를 종료시킨다. 
	// kill 은 하나의 인자를 가진다. 인자는 소켓지정번호다.
	while(1)
	{
		FD_ZERO(&fd_w);
		FD_SET(0,  &fd_w);
		FD_SET(server_sockfd, &fd_w);

		select(server_sockfd + 1, &fd_w, (fd_set *)0, (fd_set*)0, NULL); 
		if(FD_ISSET(server_sockfd,&fd_w))
		{
			client_sockfd = accept(server_sockfd, (struct sockaddr *)&clientaddr,
											(socklen_t *)&client_len);
			ClientClass = new tClass(client_sockfd);
			Cinfo[client_sockfd] = ClientClass->tClassId();
		}
		else if(FD_ISSET(0, &fd_w))
		{
			memset(buf, 0x00, MAXLEN);
			read(0, buf, MAXLEN);
			if(strncmp(buf, "show", 4) == 0)
			{
				map<int, pthread_t>::iterator mi;
				mi = Cinfo.begin();
				printf("NUM\tThread ID\tClient Address\n");
				while(mi != Cinfo.end())
				{
					getsockname(mi->first, (struct sockaddr *)&myaddr, (socklen_t *)&client_len);
					printf("%d\t%u\t%s\n", mi->first, mi->second, inet_ntoa(myaddr.sin_addr));
					*mi++;
				}
				printf("=============================\n");
			}
			if (strncmp(buf, "kill", 4) == 0)
			{
				map<int, pthread_t>::iterator mi;
				mi = Cinfo.find(atoi(buf+5));
				if (mi == Cinfo.end())
				{
					printf("Not Found Thread\n");
				}
				else
				{
					printf("Kill Thread %u\n", mi->second);
					pthread_cancel(mi->second);
				}
			}
		}
		else
		{
			printf("Unknown\n");
		}
	}

}
~~~

# Thread Exception 처리

특별히 자바에서 isInterrupted() 란 메소드는 쓰레드에서 InterruptedException 이 발생하고 이를 **catch하지 않고 그냥 종료되었을때 true를 리턴하고 catch하였을때는 false를 리턴함**을 주목하자. 그리고 이 클래스에서도 동일하게 동작한다. 또한, 한번 종료된 쓰레드는 다시 start시킬 수 없으며 이 또한 동일하게 구현되었다

~~~
Thread.h

#ifndef THREAD_H_
#define THREAD_H_

#include <pthread.h>
#include <string>
#include <stdexcept>

using namespace std;

	
class Thread
{
public:
	Thread();
	Thread(const string& name);
	virtual ~Thread();
	void start();
	void join();
	void join(unsigned long time);
	void interrupt();
	bool isInterrupted();
	bool isAlive();
	const string& getName();
	void setName(const string& name);
	
private:
	int _state;
	bool _joinning;
	bool _isInterrupted;
	pthread_mutex_t _mutex;
	pthread_cond_t _cond;
	static pthread_attr_t _attr;
	pthread_t _thread;
	static long _id;
	string _name;
	static void* threadHandler(void* arg);
	static void interruptHandler(int sigInt);
	void init(const string& name);
	virtual void run() = 0;
	
};

class InterruptedException : public exception 
{
public:
	explicit InterruptedException(const string&  message)
	: _message(message)
	{}

	virtual ~InterruptedException() throw()
	{}

	virtual const char* what() const throw()
	{
		return _message.c_str();
	}

private:
	string _message;
	
};


class IllegalThreadStateException : public exception 
{
public:
	explicit IllegalThreadStateException(const string&  message)
	: _message(message)
	{}

	virtual ~IllegalThreadStateException() throw()
	{}

	virtual const char* what() const throw()
	{
		return _message.c_str();
	}

private:
	string _message;
	
};

#endif /*THREAD_H_*/


Thread.cpp

#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>
#include <signal.h>
#include "Thread.h"

using namespace std;


long Thread::_id = 0;

pthread_attr_t Thread::_attr;

void* Thread::threadHandler(void* arg)
{
	signal(SIGUSR1, interruptHandler);
	Thread* thread = (Thread*) arg;
	try
	{
		thread->run();
	}
	catch (InterruptedException& e)
	{
		thread->_isInterrupted = true;
	}
	catch (...)
	{
	}
	thread->_state = -1;
	if (thread->_joinning)
	{
		pthread_mutex_lock(&thread->_mutex);
		pthread_cond_broadcast(&thread->_cond);
		pthread_mutex_unlock(&thread->_mutex);
	}
	return NULL;
}

void Thread::interruptHandler(int sigInt)
{
	throw InterruptedException("Thread interrupted");	
}

Thread::Thread()
{
	char name[32];
	sprintf(name, "Thread-%ld", _id);
	init(name);
}

Thread::Thread(const string& name)
{
	init(name);
}

Thread::~Thread()
{
	pthread_attr_destroy(&_attr);
	pthread_mutex_destroy(&_mutex);
	pthread_cond_destroy(&_cond);
}

void Thread::init(const string& name)
{
	_name = name;
	_joinning = false;
	_isInterrupted = false;
	pthread_mutex_init(&_mutex, NULL);
	pthread_cond_init(&_cond, NULL);
	_state = 0;
	pthread_mutex_lock(&_mutex);
	if (_id == 0)
	{
		pthread_attr_init(&_attr);
		pthread_attr_setdetachstate(&_attr, PTHREAD_CREATE_DETACHED);
	}
	_id++;	
	pthread_mutex_unlock(&_mutex);
}

void Thread::start()
{
	if (_state == 1)
	{
		throw IllegalThreadStateException("Thread already started");
	}
	else if (_state == 0)
	{
		_state = 1;
		pthread_create(&_thread, &_attr, threadHandler, this);
	}
	else if (_state == -1)
	{
		throw IllegalThreadStateException("Thread had been started");
	}
}

void Thread::join()
{
	if (_state == 1)
	{
		pthread_mutex_lock(&_mutex);
		_joinning = true;
		pthread_cond_wait(&_cond, &_mutex);
		_joinning = false;
		pthread_mutex_unlock(&_mutex);
	}
}

void Thread::join(unsigned long time)
{
	if (_state == 1)
	{
		struct timeval now;
		struct timespec timeout;
		gettimeofday(&now, NULL);
		ldiv_t t = ldiv(time * 1000000, 1000000000);
		timeout.tv_sec = now.tv_sec + t.quot;
		timeout.tv_nsec = now.tv_usec * 1000 + t.rem;
		pthread_mutex_lock(&_mutex);
		_joinning = true;
		pthread_cond_timedwait(&_cond, &_mutex, &timeout);
		_joinning = false;
		pthread_mutex_unlock(&_mutex);
	}
}

void Thread::interrupt()
{
	pthread_kill(_thread, SIGUSR1);
}

bool Thread::isInterrupted()
{
	return _isInterrupted;
}

bool Thread::isAlive()
{
	return _state == 1;
}

const string& Thread::getName()
{
	return _name;
}

void Thread::setName(const string& name)
{
	_name = name;
}


Test

#include <iostream>
#include "Thread.h"

using namespace std;


class Test1 : public Thread
{
private:
	void run()
	{
		string name = getName();
		for (int i = 0; i < 10; i++)
		{
			try
			{
				cout << name << endl;
				sleep(1);
			}
			catch (InterruptedException& e)
			{
				cout << name << ": " << e.what();
				cout << " but continue" << endl;
			}
		}
		cout << name << " ending" << endl;
	}
};


class Test2 : public Thread
{
private:
	void run()
	{
		string name = getName();
		for (int i = 0; i < 10; i++)
		{
			cout << name << endl;
			sleep(1);
		}
		cout << name << " ending" << endl;
	}
};

class Test3 : public Thread
{
private:
	Thread& _t;
	
	void run()
	{
		cout << getName() << " is joinning " << _t.getName() << endl;
		_t.join();
		cout << getName() << " is ending" << endl;
	}
	
public:
	Test3(Thread& t)
	: _t(t) {}
};

void checkAlive(Thread& thread)
{
	string name = thread.getName();
	if (thread.isAlive())
	{
		cout << name << " is alive" << endl;
	}
	else
	{
		cout << name << " is died" << endl;
	}
}


void checkInterrupt(Thread& thread)
{
	string name = thread.getName();
	if (thread.isInterrupted())
	{
		cout << name << " is interrupted" << endl;
	}
	else
	{
		cout << name << " is not interrupted" << endl;
	}
}

int main()
{
	try
	{
		Test1 t1;
		Test2 t2;
		t2.setName("Second Thread");
		string t1Name = t1.getName();
		string t2Name = t2.getName();
		t1.start();
		t2.start();
		sleep(2);
		cout << "Now joinning " << t1Name << " until 2 sec" << endl;
		t1.join(2000);
		checkAlive(t1);
		checkAlive(t2);
		try
		{
			cout << "Now tring restart " << t1Name << endl;
			t1.start();
		}
		catch (exception& e)
		{
			cerr << typeid(e).name() << ": " << e.what() << endl;
		}
		sleep(2);
		cout << "Now interrupt " << t1Name << endl;
		cout << "Now interrupt " << t2Name << endl;
		t1.interrupt();
		t2.interrupt();
		sleep(2);
		checkAlive(t1);
		checkAlive(t2);
		cout << "Now joinning " << t2Name << endl;
		t2.join();
		Test3 t3(t1);
		t3.setName("Third Thread");
		t3.start();
		sleep(1);
		cout << "Now joinning " << t1Name << endl;
		t1.join();
		checkAlive(t1);
		checkAlive(t2);
		checkAlive(t3);
		checkInterrupt(t1);
		checkInterrupt(t2);
		checkInterrupt(t3);
		sleep(1);
		cout << "Now restart " << t2Name << endl;
		t2.start();
	}
	catch (IllegalThreadStateException& e)
	{
		cerr << typeid(e).name() << ": " << e.what() << endl;
	}
	sleep(1);
	cout << "Main thread ending" << endl;
	return 0;
}

~~~

# Thread 취소와 종료

## Thread 취소(cancellation)와 종료
쓰레드는 제어가능한 객체로 필요에 따라 생성시킬 수 있듯이 필요에 따라서 중단 시킬 수도 있다. 이 쓰레드 중단이라는 것이 매우 단순한 행위라고 생각되지만 생각처럼 그렇게 단순한 행위가 아니다.
문서는 쓰레드 취소에 관련된 내용과 쓰레드 종료시 신경써야될 (자원정리 와 같은)것들에 대해서 알아보도록 한다.

## Thread 취소
멀티 쓰레드 프로그램에서 특정 쓰레드를 중단 시키고자 할때를 위해서 Pthread는 ptread_cancel()이라는 함수를 제공한다.
**int pthread_cancel(pthread_t thread);**
이 함수는 인자로 주어진 쓰레드 식별번호 thread를 가지는 쓰레드를 중지시킨다. 명확히 말하자면 쓰레드를 중지 시키는게 아니고 쓰레드에 취소 요청을 하는 것으로 봐야 한다. 취소 요청을 받은 쓰레드가 어떻게 반응 할런지는 요청을 받은 쓰레드의 취소 상태 설정에 의존한다. 취소 요청을 받은 쓰레드는 취소 상태에 의해서 필요한 작업을 한 후 종료 하게 된다. 취소 요청을 받아서 종료하는 쓰레드는 pthread_exit(PTHREAD_CANCELED)를 호출하고 종료한다. pthread_cancel()에 의해서 취소가 통보된 쓰레드는 쓰레드 취소 상태의 설정에 따라서 취소 요청을 무시할 수도 취소지점(cancellation point) 지점까지 수행한뒤에 종료 될수도 있기 때문이다.
**쓰레드 취소와 종료는 엄연히 다르다는 것을 이해하기 바란다. 그렇지 않으면 앞으로 문서의 내용을 읽는데 헛갈릴 수 있다.**

## 쓰레드 취소상태의 설정
쓰레드가 pthread_cancel()에 의해서 취소요청을 받았을 때 어떻게 반응할런지를 결정하는 쓰레드 취소상태는 여러가지 방법에 의해서 결정된다. 취소 상태는 pthread_setcancelstat() 함수에 의해 결정한다.

**int pthread_setcancelstate(int state, int *oldstate);**
첫번째 인자인 state는 새로운 취소상태를 설정하기 위해서 사용된다. 두번째 인자인 oldstate는 이전의 취소상태의 설정값을 받아오기 위해서 사용된다. 이 함수는 다음과 같이 사용할 수 있다.
- int old_cancel_state;
- pthread_setcancelstate(PTHREAD_CANCEL_DISABLE, &old_cancel_state);
- pthread_setcancelstate(PTHREAD_CANCEL_ENABLE, &old_cancel_state);
만약 이전의 취소상태 설정값이 필요 없다면 old_cancel_state대신 NULL을 사용하면 된다. 위의 사용에서 처럼 쓰레드는 PTHREAD_CANCEL_DISABLE와 PTHREAD_CANCEL_ENABLE 둘중 하나의 취소상태를 가질 수 있다. **만약 PTHREAD_CANCEL_ENABLE 상태라면 쓰레드는 취소요청을 받아들이고 취소지점까지 진행한다음 취소 지점을 벗어나야지 종료한다. ENABLE 상태일 경우 별도로 취소지점까지 진행한다음 종료할 것인지 아니면 바로 종료할 것인지를 pthread_setcanceltype() 를 통해 지정**할 수 있다. 만약 **PTHREAD_CANCEL_DISABLE 상태라면 취소요청을 받은 후 취소지점까지의 진행을 하지 않고 바로 종료**한다.

## 쓰레드 취소종류의 설정
쓰레드 취소상태가 PTHREAD_CANCEL_ENABLE인 경우 취소의 종류를 결정할 수 있다. 취소종류의 결정은 pthread_setcanceltype()을 통해서 이루어진다.
**int pthread_setcanceltype(int type, int *oldtype);**
취소종류는 type를 통해서 결정된다. **PTHREAD_CANCEL_ASYNCHRONOUS와 PTHREAD_CANCEL_DEFERRED** 둘 중 하나를 선택할 수 있다. **전자일 경우 바로 종료**하며, **후자의 경우 취소지점을 벗어날 때까지 기다리게 된다**. oldtype를 이용해서 이전 취소타입을 얻어 올 수 있다. NULL이라면 받아오지 않는다. 이 함수는 당연하지만 취소상태가 PTHREAD_CANCEL_DISABLE 라면 의미 없는 함수다.

## 취소지점
**쓰레드에게 취소요청이 왔다고 해서 무조건 취소해 버리면 문제가 생길 수도 있다. 어떤 일을 처리하고 있는 중에 취소요청이 전달했는데, 별로 중요하지 않는 (무시해도 될만한) 일이라면 중단후 바로 취소해도 되겠지만 중요한 일을 처리하는 중이라면 일을 처리한후 종료 해야** 할것이다. 이 마지 노선이 취소지점이다.
취소지점으로 설정될 수 있는 영역은 다음과 같다.
- pthread_join(3)
- pthread_cond_wait(3)
- pthread_cond_timedwait(3)
- pthread_testcancel(3)
- sem_wait(3)
- sigwait(3)

pthread_setcancelstate()함수에 의해서 PTHREAD_CANCEL_ENABLE 상태로 되어 있다면 취소지점을 무시하고 즉시 종료 된다. PTHREAD_CANCEL_DISABLE로 되어 있다면 위의 취소지점을 벗어날 때까지 기다린다. 즉 **취소요청을 받은 쓰레드가 pthread_cond_wait()에서 조건변수를 기다리는고 있다면 조건변수 를 받을 때까지 취소를 유보**하게 된다.

## 쓰레드 기본 취소상태와 취소 종류
별다른 설정이 없을 경우 pthread_create()로 만들어 지는 쓰레드는 PTHREAD_CANCEL_ENABLE, PTHREAD_CANCEL_DEFERRED로 상태로 생성된다.

## 쓰레드 종료시 자원정리
thread_cancel()등을 통해서 종료 통보를 받은 쓰레드는 종료하기 전에 여러가지 일을 해주어야 할 것이다. 뭐 간단한 쓰레드라면 관계 없겠지만 **복잡하게 얽혀 있는 멀티 쓰레드 프로그램이라면 이런 저런 정리**해줘야 할 것들이 많을 것이다.

## 쓰레드 종료시 자원해제
쓰레드에서 **malloc()등을 호출해서 메모리 공간**을 확보했다거나 **DB나 파일, 소켓등을 열어서 작업했다면 반드시 이들 자원을 해제**시켜줘야 한다. 간단하게 생각하자면 **쓰레드 종료시점에서 free(), close(), DB라면 이런 저런 정리**를 해주면 될것이다. 그러나 pthrad_cancel()등에 의해서 작업중간에 요청을 받았다면 그리 간단한 문제가 아니다. 쓰레드 마지막까지 루틴을 진행할 수 없기 때문이다. 이럴 경우를 대비해서 **pthread_cleanup_push(), pthread_cleanup_pop()와 같은 함수를 제공**한다.
이 **함수들을 이용해서 쓰레드가 종료할 때 호출해야할 함수를 지정**할 수 있다. 그러므로 프로그래머는 이들 함수에 **자원해제와 같은 필요한 코드**를 넣어두기만 하면 된다. 

~~~
#include <pthread.h>
#include <unistd.h>
#include <stdlib.h>
#include <stdio.h>
#include <sys/time.h>

// 쓰레드 종료시 호출될 함수
void clean_up(void *);

// 쓰레드 함수
void *thread_func(void *);

pthread_cond_t cond = PTHREAD_COND_INITIALIZER;
pthread_mutex_t lmu = PTHREAD_MUTEX_INITIALIZER;

int main(int argc, char **argv)
{
    pthread_t pt;
    pthread_create(&pt, NULL, thread_func, NULL);

    // 생성된 쓰레드 pt에 취소 요청을 보낸다.
    pthread_cancel(pt);

    // 5초를 쉰 후에 시그널을 보낸다. 
    sleep(5);
    pthread_cond_signal(&cond);

    // join후 종료한다.
    pthread_join(pt, NULL);
    printf("exit\n");
    exit(1);
}

// 쓰레드 종료시 효출될 함수 
// 여기에 자원해제루틴을 입력할 수 있을 것이다.
void clean_up(void *arg)
{
    printf("Thread cancel Clean_up function\n");
}

void *thread_func(void *arg)
{
    // DISABLE 상태다. 
    // 이경우 쓰레드에 대해서 취소 요청을 무시한다.   
    pthread_setcancelstate(PTHREAD_CANCEL_DISABLE, NULL);

    // 쓰레드 종료시 호출될 함수 등록
    pthread_cleanup_push(clean_up, (void *)NULL);

   
    pthread_mutex_lock(&lmu);
    printf("THREAD cond wait\n");
    pthread_cond_wait(&cond, &lmu);
    printf("NO WAIT COND\n");
    pthread_mutex_unlock(&lmu);

    printf("EXIT\n");
    pthread_cleanup_pop(0);
}
~~~

main 쓰레드는 thread_func를 실행시킨다. thread_func는 pthread_cond_wait()에서 신호가 발생하기를 기다리게 된다. 잠시후 main 쓰레드는 pthread_cancel()을 이용해서 thread_func 쓰레드에 취소 요청을 하게 된다. 위의 경우 thread_func쓰레드는 취소상태가 PTHREAD_CANCEL_DISABLE로 되어 있기 때문에 요청을 무시한다. 때문에 cleanup함수도 실행되지 않는다.

# 쓰레드 메시지 전달

## 소개
쓰레드 프로그래밍을 할 때 가장 신경쓰이는건 역시 쓰레드동기화(:12)와 쓰레드간 메시지 전달과 관련된 문제일 것이다. 또한 쓰레드간 메시지 전달에는 쓰레드 동기화 문제까지 함께 고민해야 한다. 이 문서는 다중쓰레드에서 쓰레드간 메시지를 효과적으로 전달하기 위한 다양한 방법들을 기술한다.
여기에서 소개하는 방법들은 수많은 방법들 중 몇가지 방법들일 뿐이다. 실제 프로젝트에서는 다양한 응용을 생각해야 할 것이다.

## 시나리오
영어문서를 파싱해서 Term을 얻어오고, 출현한 Term의 빈도수를 계수하는 프로그램을 만들도록 하겠다. 빠른 파싱을 위해서, 문서가 주어지면 문서를 라인수를 기준으로 4등분 한다음, 4개의 쓰레드를 돌려서 병렬로 처리하도록 할 것이다. 이 프로그램은 다음의 사항을 만족시켜야 한다.
1. Main 쓰레드는 문서를 4등분한다음 생성된 work thread갯수를 파악한다음
2. 파일을 open()한 후, 4개의 쓰레드에게 읽어야될 파일지정자와 파싱할 줄의 범위를 알려준다.
3. 파일지정자와 범위를 전달받은 work thread는 해당범위의 문장을 분석해서 <Term, count> 자료구조를 만든다.
4. 모든 작업이 끝났다면, Main Thread에게 작업이 끝났음을 알려준다. 이때, 자신이 작업한 결과에 대한 정보를 Main Thread에게 알려줘야 한다.
5. 모든 work Thread에서 작업종료 메시지를 받았다면, Main Thread는 각 work Thread의 <Term, count>를 취합해서, 하나의 파일로 만든다.

## 구현방안
다음은 문서를 파싱해서 <Term, count>를 얻어오는 프로그램이다. 단일 쓰레드로 작동하는 프로토타입의 프로그램으로 아래의 코드를 멀티쓰레드방식으로 수정할 것이다.

~~~
#include <sys/types.h>
#include <regex.h>
#include <string.h>
#include <stdio.h>
#include <errno.h>
#include <vector>
#include <string>
#include <iostream>

#include <fcntl.h>

using namespace std;

/*
 * 주어진 문서에서 단어를 얻어온다.
 * 입력된 라인은 strtok를 통해서 토큰으로 먼저 분리되고
 * 분리된 문자열은 정규표현()을 만족하면 Term으로 판단하고 출력한다.
 * 실행인자 : 정규표현식
 * 사 용 예 :
 */

int main(int argc, char **argv)
{
  FILE *fp;
  int rtv;
  regex_t preg;
  char linebuf[1024];
  char *tr;
  char seps[] = " -.,()\";:{}'+@/<>[]|!?#";
  char msg[64];
  int i;
  vector<string> FileList;

  // 파싱할 파일을 등록한다.
  FileList.push_back("rfc.dat");

  if (argc != 2)
  {
    printf("Usage : %s [pattern]\n", argv[0]);
    return 1;
  }

  // 정규표현을 위한 컴파일러 생성
  rtv = regcomp(&preg, argv[1], REG_EXTENDED|REG_NOSUB);
  if(rtv != 0)
  {
    regerror(rtv, &preg, NULL, 0);
    return 1;
  }

  for(int i = 0; i < FileList.size(); i++)
  {
    fp = fopen(FileList[i].c_str(), "r");
    if (fp == NULL)
    {
      perror("Error ");
      return 1;
    }

    while(fgets(linebuf, 1024, fp) != NULL)
    {
      linebuf[strlen(linebuf)-1] = '\0';

      // 토큰으로 구분한다음
      tr = strtok(linebuf, seps);
      while(tr != NULL)
      {
        tr = strtok(NULL, seps);
        if (tr != NULL)
        {
          // 정규표현을 만족하는지 확인한다.
          if (regexec(&preg, tr, 0, NULL, 0) == 0)
          {
            cout << "Find Term : " << tr << endl;
          }
        }
      }
    }
    fclose(fp);
  }
}

./getterm "[a-zA-Z0-9]"
~~~

## 메시지큐 구현
메시지큐 구현에 대한 아이디어는 다음과 같다.
- worker Thread가 하나의 Term을 얻어오면, 해당 Term을 메시지 형태로 Main Thread에게 바로 전달한다.
- Main Thread는 <Term, count>자료구조를 유지하면서, 계수를 한다.
- 메시지큐의 ID는 하나로 통일한다.
- 메시지큐를 이용한 메시지의 전달 가능성 확인을 목적으로 한다.(병렬성 효율등은 부차적 문제로 언급한다)

### worker Thread 관리
주고 받는 데이터에 타입을 두어서 관리하도록 할 것이다
~~~
struct Data
{
	int type;
	char *Data;
	int size;
};
~~~

Type는 다음과 같이 정의 할 것이다
1 << 1	일반 데이터
1 << 2	제어 데이터
- 일반데이터라면 Data를 파싱된 Term으로 인식해서 처리한다.
- 제어데이터라면 Data를 제어 명령이라고 인식해서 처리한다. Data가 1 이라면, 모든 일을 끝냈다고 판단한다.

이 방식은 구현이 단순하긴 하지만 worker thread -> Main thread로의 단방향 데이터 전송만 가능하다는 단점이 있다. 메시지큐()를 하나더 만드는등 다양한 방법이 있을 수 있는데, 이는 뒤에서(몇가지만) 다루도록 하겠다.

### 쓰레드 동기화
쓰레드간 동기화는 mutex() 잠금과 조건변수()를 이용할 것이다.

### 프로시져
~~~
main
{
	생성 쓰레드 갯수는 4개로 한다.
	문서를 읽어들여서 문서의 Line수를 계수한다.
	메시지큐(:12)를 생성한다.
	for(i = 0; i < 4; i++)
	{
		// 쓰레드 동기화 Start
		Worker 쓰레드를 생성한다. 인자로 문서의 Offset정보를 넘긴다.
		쓰레드 함수명은 WThread로 한다. 
		// 쓰레드 동기화 End 
	}
	메인 쓰레드를 수행한다.
	while(1)
	{
		메시지 큐로부터 데이터를 읽는다.
		switch(데이터 타입)
		case 일반데이터
		{
			Term을 계수한다.
		}
		case 제어데이터
		{
			종료루틴을 수행한다.
			모든 쓰레드가 작업을 종료했다면, Break; 
		}
	}
	<Term,Count>결과를 출력한다.
}
~~~

### 코드 구현
<!> 미완성 코드다. g++ 로 컴파일 하면, 메시지큐를 통해서 worker thread에서 main thread로 분석된 Term이 전달되는걸 확인할 수 있다. 메시지큐를 통한 데이터 통신의 기본적인 구현은 끝났다고 볼 수 있다. 코드를 좀더 깔끔하게 하고, 몇 군데 예외처리를 해주어야 한다.
~~~
#include <pthread.h>
#include <unistd.h>
#include <stdlib.h>
#include <stdio.h>
#include <sys/types.h>
#include <sys/ipc.h>
#include <sys/msg.h>
#include <sys/stat.h>
#include <regex.h>

#include <fcntl.h>
#include <string.h>

#define ThreadNum 4

pthread_mutex_t mutex_lock;
pthread_cond_t  sync_cond;

pthread_mutex_t gmutex = PTHREAD_MUTEX_INITIALIZER;
pthread_cond_t  gcond  = PTHREAD_COND_INITIALIZER;

const char seps[] = " -.,()\";:{}'+@/<>[]|!?#\n"; // Token

// 주고 받을 데이터
struct Data
{
	int queuenum;
	int msgtype; 
	int tid;
	char msg[256];
	int size;
	key_t key_id;
};

// worker 쓰레드에 넘겨줄 정보
struct DocInfo
{
	int tnum;       // thread id
	int start;      // 문서시작 위치
	int end;        // 문서 끝 위치
	char fname[80]; // 작업 파일명
	char regex[24]; // 정규표현 문자열 
};

/*
 * worker 쓰레드 함수
 */
void *Tfunction(void *data)
{
	struct Data lData;
	struct DocInfo *lDocInfo; 
	FILE *fp;
	char line[256];
	int rtv;
	int bsize;
	int readn = 0;
  regex_t preg;
  char *tr;

	lData = *(struct Data *)(data);
	lDocInfo = (struct DocInfo *)lData.msg; 

	bsize = lDocInfo->end - lDocInfo->start;
	printf("DEBUG Thread %d %d\n", lDocInfo->tnum, bsize);
	if((fp = fopen(lDocInfo->fname, "r")) == NULL)
	{
		perror("Fopen Error");
		exit(0);
	}

	lseek(fileno(fp), lDocInfo->start, SEEK_SET);	
  rtv = regcomp(&preg, lDocInfo->regex, REG_EXTENDED|REG_NOSUB);

	pthread_mutex_lock(&mutex_lock);
	pthread_cond_signal(&sync_cond);
	pthread_mutex_unlock(&mutex_lock);

	if (rtv != 0)
	{
		regerror(rtv, &preg, lDocInfo->regex, REG_EXTENDED|REG_NOSUB);
		exit(0);
	}
	for (readn = 0; readn < bsize; )
	{
		if(fgets(line, 256, fp) == NULL)
			break;
		tr = strtok(line, seps);
		if (tr != NULL)
		{
			if (regexec(&preg, tr, 0, NULL, 0) == 0)
			{
				lData.queuenum = 1;
				lData.msgtype = 1 << 1;
				lData.size = strlen(tr);
				sprintf(lData.msg, "%s", tr);

				if(msgsnd(lData.key_id, (void *)&lData, sizeof(lData), IPC_NOWAIT) <0)
				{
					perror("msg snd error");
					exit(0);
				}
			}
		}
		readn += strlen(line);
		sleep(1);
	}
	// 종료 메시지를 보낸다.
}


int main(int argc, char **argv)
{
	struct Data lData;
	struct DocInfo lDocInfo; 

	pthread_t p_thread[ThreadNum];

	int fd;
	char *fname;
	int fsize = 0;
	int blocksize = 0;
	struct stat fileinfo;
	char *regex;

	// Message queue
	key_t key_id;
	int msgtype;	

	fname = argv[1];
	regex =  argv[2];

	// 메시지큐 생성
	key_id = msgget((key_t)8888, IPC_CREAT|0666);
	if (key_id == -1)
	{
		perror("msgget error : ");
		exit(0);
	}

	// Open File
	fd = open(fname, O_RDONLY);	
	if (fd < 0)
	{
		perror("Open file error");
		return 1;
	}
	if(fstat(fd, &fileinfo)< 0) 
	{
		return 1;
	}
	fsize = fileinfo.st_size;
	printf("T Size is %d\n", fsize);

	blocksize =  fsize / ThreadNum; 

	for (int i = 0; i < ThreadNum; i++)
	{
		lDocInfo.start = (i*blocksize);
		lDocInfo.end = lDocInfo.start+blocksize;
		sprintf(lDocInfo.fname, "%s", fname);
		sprintf(lDocInfo.regex, "%s", regex);
		lDocInfo.tnum = i;

		if (i == (ThreadNum -1))
			lDocInfo.end += fsize%ThreadNum;
		lData.queuenum = 1; 
		lData.msgtype = 1 << 2; 
		lData.size = sizeof(lDocInfo); 
		lData.key_id = key_id;
		lData.tid = lDocInfo.tnum; 
		memcpy((void *)lData.msg, (void *)&lDocInfo, sizeof(lDocInfo));

		pthread_mutex_lock(&mutex_lock);
		pthread_create(&p_thread[i], NULL, Tfunction, (void *)&lData);
		pthread_cond_wait(&sync_cond, &mutex_lock);
		pthread_mutex_unlock(&mutex_lock);
	}

	while(1)
	{
		if(msgrcv(key_id, (void *)&lData, sizeof(lData), (1 >> 1), 0)	== -1)
		{
			perror("msgrcv error: ");
		}
		printf("Get Term %d %s\n", lData.tid, lData.msg);
	}
}
~~~

## 공유메모리 구현
공유메모리()는 커널에 메모리 공간을 할당함으로써, 시스템 전역적으로 자원을 사용할 수 있도록 지원하는 IPC() 설비중 하나다. 이러한 특성을 이용 쓰레드간 메시지 교환은 공유메모리()상에 환형큐()를 만드는 것으로 구현가능할 수 있을 것이다.
이 환형큐에는 여러개의 쓰레드가 접근을 하게 될 것이므로, 쓰레드간 동기화가 필수 적일건데, mutex()가 아닌 레코드:::잠금()으로도 쓰레드간 접근제어를 할 수 있다. **레코드 잠금을 통해서 쓰레드동기화를 제어하는 방법은 공유메모리와 파일잠금을 이용한 프로세스간 데이터 공유**에서 이미 다룬바가 있다.
쓸만한 예제라고 생각은 하지만 이번 구현에 그대로 적용하기에는 성격이 다르다. 이번 구현은 여러개의 생산자와 하나의 소비자가 존재하는 형식이기 때문이다. 뭐 그렇다고 크게 문제될건 없다. 아래와 같이 생산/소비자 방식을 약간 수정하는 것으로 해결책을 삼았다.

**쓰레드간 동기화는 그대로 파일의 레코드 잠금**을 이용할 것이다. 이 경우 다수의 생산자를 제어해야 하는데, 처음 8 byte()를 생산자간 접근제어를 위한 잠금으로 사용할 것이다. **처음 레코드에 대한 생산자만이 레코드에 큐(빨간색)에 쓸권한**을 가지게 된다.
**데이터 쓰기 권한을 얻은 쓰레드는 공유메모리에 데이터를 쓰고, 해당 공유메모리에 대응되는 레코드에 잠금을 풀게**된다. 그러면 **소비자는 잠금을 얻고, 잠금에 대응되는 공유메모리를 찾아가서 정보**를 읽어오면 된다. 예제로 제시한 문서를 이해하고 있다면, 위의 방식으로 수정하는건 그리 어렵지 않을 것이니, 굳이 코드를 만들진 않도록 하겠다.


# 쓰레드 우선순위 문제 해결
"쓰레드 우선순위"에 따라서 발생하는 문제점에 대해서 알아보겠다. 이 문제는 주의해서 프로그래밍 하지 않을 경우 간혹 발생하기도 한다.

## 문제 발생
~~~
#include <pthread.h>
#include <sys/wait.h>
#include <string.h>
#include <stdlib.h>
#include <unistd.h>
#include <stdio.h>

void *threadfunc(void *arg);

main()
{
    int n, i, j;
    pthread_t threadid;

    if ((n = pthread_create(&threadid, NULL, threadfunc, NULL)) != 0 )
    {
        perror("Thread create error ");
        exit(0);
    }
    printf("Main Thread START\n");
    for (i = 0; i < 100000000; i++)
    {
        j = i*5;
    }
    printf("Main Thread END\n");
    pthread_join(threadid, NULL);
}

void *threadfunc(void *arg)
{
    int i, j;
    printf("Thread Start\n");

    for (i = 0; i < 100000000; i++)
    {
        j = i*5;
    }

    printf("Thread:end\n");
    return ;
}
~~~
리눅스에서 처럼 2개의 쓰레드가 동시에(정확히는 동시가 아니지만 이해하기 쉽게)수행 되는게 아니고 메인 쓰레드가 끝날 때까지 쓰레드 생성이 되지 않음을 알 수 있다. 코드에 따라서는 특정 쓰레드가 영원히 실행되지 않는 문제가 발생할 수도 있다. 리눅스에서 개발되는 많은 프로그램들이 다른 유닉스(:12)로도 배포 되는 것을 감안한다면 이는 매우 심각한 문제이다.
솔라리스에서 이러한 문제가 발생하는 이유는 쓰레드라 할지라도 개별적인 프로세스로 실행되는 **리눅스와 달리 하나의 LWP를 나눠서 사용하기 때문에 하나의 쓰레드에서 매우 바쁘게 작동할경우 다음 쓰레드 생성을 위한 스케쥴링에 문제**가 생기기 때문이다.

## 문제 해결
비록 위의 문제가 리눅스에서 발생하진 않지만 많은 리눅스에서 개발되는 프로그램이 솔라리스등의 다른 유닉스로 포팅되고 있다는 것을 감안한다면 반드시 잡아줘야 할 문제다.

문제의 해결방법은 비교적 간단한데, **sleep()계열의 함수를 이용**해서 쓰레드와 쓰레드 사이에 약간의 시간간격을 두어서 쓰레드 스케쥴링을 할 수 있도록 시간을 벌여주면 된다. 다음은 문제를 해결한 코드이다.
~~~
  if ((n = pthread_create(&threadid, NULL, threadfunc, NULL)) != 0 )
  {
      perror("Thread create error ");
      exit(0);
  }
  usleep(100);
  printf("Main Thread START\n");
		
~~~

## 결론
어떻게 보면 **팁수준의 매우 간단한 내용인것 같지만 이런 문제일수록 문제의 원인이 애매 모호해서 해결책을 찾기 어려운 경우가 많다**. 이런 경우도 마찬가지다. 아마도 pthread:::라이브러리()의 (그리 심각하지 않은?)문제라고 생각되는데 막상 문제가 발생했을 때는 정말 사람을 짜증나게 할 수 있는 문제다.

# Pthread - 더 깊이
다중 쓰레드 프로그래밍시 필요로하는 Condition:::variables() 를 통한 좀더 정교한 쓰레드간 동기화 방법에 대해서 다루게 된다.

## 쓰레드 종류
### User Space 과 Kernel Space 쓰레드
Pthread 는 **Posix:::Thread() 로 Posix 에서 표준으로 규격화한 쓰레드 라이브러리()이며, User:::Space() 에서 작동하는 쓰레드 라이브러리**이다. **즉 쓰레드 관리를 위해서 Kernel 이 관여를하지 않고 쓰레드 관련 라이브러리에서 대부분의 쓰레드 관리를 처리**하게 된다. **Kernel 이 직접 쓰레드를 관리하는 방식을 Kernel:::Space() 쓰레드 라고 하는데, User Space 쓰레드 보다 비용이 좀더 많이 드는 관계로 최근에는 User Space 라이브러리를 좀더 선호하는 편**이다.

Kernel Space 이 User Space 쓰레드 보다 비용이 더 많이 든다는데 의아해 할수도 있을것이다. 일반적으로 생각했을때 Kernel 은 좀더 시스템에 가까우므로 좀더 비용이 적고 빠를것 이라고 생각할수 있지만 실은 그렇지 않다. 보통 응용프로그램은 2가지 레벨에서 작동하게 된다. 하나는 Kernel Space 모드 이고 다른 하나는 User Space 모드이다. 이렇게 2가지 모드로 나누는 이유는 일반응용 프로그램이 시스템에 직접 접근하는 일을 막기 위함으로, 일반 응용 프로그램이 시스템에 접근하기 위해서는 **System:::Call() 을 불러서 Kernel 에게 시스템자원을 사용할것을 요청하는 방식으로 간접 접근**해야 한다. 이때 **커널은 보호모드 상태에서 System Call 을 처리**해야 하므로 여러가지 연산을 가지게 되된다. 최종적으로 **User 모드에서 Kernel 모드로 전환하는데 드는비용, 보호모드를 위한 여러가지 연산을 하는데 드는 비용**등 3가지의 비용을 소비하게 된다.

반면 **User Level 쓰레드는 대부분의 작업을 system call 을 사용하지 않고 사용자라이브러리 에서 대부분 처리하므로 Kernel Level 쓰레드가 가지는 비용을 줄일수** 있다. 단점이라면 **Kernel Level 쓰레드가 각 쓰레드를 보호해주는 반면 유저레벨 쓰레드는 보호를 받을수 없다는 점**이다. 그러므로 **유저레벨쓰레드 사용시에는 각 쓰레드를 보호하기 위한 여러가지 동기화 장치등을 사용**해야 하며, 이것은 때때로 **세밀하고 복잡한 코딩 기법을 요구**한다.

### User Space쓰레드
**쓰레드를 유지하기 위해서는 여러가지 정보**를 필요로 한다. 이러한 정보는 **"프로그램 코드", Data, Stack, File I/O, Signal 테이블 등 크게 5가지**이다. **User Space 쓰레드는 이러한 정보를 커널에서 관리하도록 하지 않고 쓰레드 라이브러리에서 직접 처리**하게 된다. 또한 **각 쓰레드간 협력이 가능하도록 switch 기능을 제공하는데, 커널쓰레드보다 일반적으로 빠른 switch 기능을 제공**한다.

단점은 **SMPs(Symmetric MultiProcessor system)의 장점들을 이용할수 없다는 것**이다. 즉 **dual 혹은 quad 의 cpu를 사용하고 있는 시스템이라 할지라도, User Space 쓰레드로는 진정한 멀티쓰레드 프로그래밍()을 할수 없다는 것이다. 오직 하나의 CPU()에서만 작동할수 있을뿐**이다. 그러므로 설사 **CPU가 2개 혹은 4개 있다고 하더라도, 수행속도차이는 그리 크지 않게 된다**. 또한 **하나의 쓰레드에서 I/O 봉쇄가 일어나게 되면 다른 모든 쓰레드까지 봉쇄**되게 된다.

물론 이러한 문제들을 위한 해법역시 존재한다. I/O 봉쇄를 아예 라이브러리 차원에서 해결할수 있도록 도와주는 쓰레드 라이브러리도 있으며, I/O 작업을 비봉쇄 모드로 작업함으로써 봉쇄를 막을수도 있다. 그리고 몇몇의 SMPs 는 **User Space 쓰레드로도 모든 SMP() 자원을 이용할수 있도록 방법을 제공**해 주기도 한다.

### Kernel Space 쓰레드
커널()에서 직접 쓰레드를 관리하며, 커널자체에서 쓰레드 정보를 가진다. 또한 각 프로세스 간의 스케쥴역시 커널에서 맡는다. 이경우 **I/O 봉쇄같은 문제는 신경쓰지 않아도 되며, 또한 SMPs 자원을 제대로 활용할수 있다는 장점**을 가진다.

## Pthread 에서의 세밀한 동기화
mutex() 는 어디까지나 잠금을 위한 것으로 동기화시킬수 있는 범위는 좁은 범위로 한정되며, mutex 만 가지고는 세밀한 수준에서의 동기화는 어렵다. 그래서 mutex 와 더불어 다른 쓰레드 도구를 사용해야 한다. **Condition variables (조건변수) 와 mutex 를 이용한 데이터:::동기화()**

### 조건변수
뮤텍스가 잠금에 의해서 소극적인 동기화를 제공한다면, 조건변수는 "시그널 발생"과 "시그널에 대한 기다림" 을 통한 좀더 적극적인 동기화 방법을 제공한다. 이러한 시그널을 기다리기 위해서 **pthread_cond_wait() 와 pthread_cond_timedwait() 2개의 함수를 제공**한다. 이 함수들은 **pthread_cond_signal()에 의해서 발생되는 시그널을 기다리고 있다가. 시그널이 도착하면 기다림을 멈추고 다음 루틴을 실행**하게 된다. 이러한 시그널의 전달은 "조건변수" 를 통해서 이루어지게 된다. 아래는 이러한 순서를 간략하게 정리한 것이다(슈도코드형식).
~~~
임계영역에 들어간다. 
{
    요청을 기다린다.(pthread_cond_wait 를 이용해서 singal이 도착하는지 확인한다) 
    만약 요청이 도착한다면 (요청에 대한 해결이 끝날때 까지 요청영역을 임계영역으로 설정한다)
    {
        요청을 해결한다. 
    }
}
~~~
임계영역을 빠져나온다. 
			
**임계영역()에 진입하고 빠져나오는 것은 mutex 를 사용한다. 위의 슈도코드를 보면 단순히 잠금만을 제공하는것이 아닌, 요청/해결 의 서버/클라이언트 모델을 구현할수 있으며, 다중의 요청/해결에 대한 동기화 문제까지 해결할수 있음을 알수 있다.**

###  조건변수 생성및 초기화
조건변수는 pthread_cont_t 의 타입으로 선언되어 있다. pthread_cond_t 는 다음과 같이 선언되어 있다.
~~~
typedef struct
{
    struct _pthread_fastlock __c_lock;  // 구조체의 잠금을 위해서 사용된다.
    _pthread_descr __c_waiting;         // 쓰레드는 이 값의 상태변화를 기다린다. 
} pthread_cond_t 
~~~				
pthread_cont_t 의 초기화를 위한 방법은 2가지가 있다. 첫번째는 프로그램의 시작시에 선언하고 초기화 해버리는 방법과, 프로그램 실행중에 함수를 호출해서 사용하는 방법이 있다. 프로그램 시작시에 선언해서 사용하기 위해서는 다음과 같은 방법을 사용한다.
**pthread_cond_t       thread_con1 =   PTHREAD_COND_INITIALIZER;**
				
PTHREAD_COND_INITIALIZER 은 {__LOCK_INITIALIZER, 0}	으로 디파인 되어 있다. 위의 초기화를 사용하게 되면 pthread_cont_t 구조체를 위의 값으로 초기화 하게 된다. 또다른 방법은 프로그램 시행중에 사용하는 방법으로 **pthread_cond_init() 함수**를 사용하면 된다.

### 조건변수에 신호 보내기
**우리는 조건변수()에 신호를 보냄으로써, 조건변수를 (신호를)기다리는 다른 쓰레드를 깨울수 있게 된다. 보통은 이러한 신호를 보내서 깨우고자 하는 쓰레드가 하나일수도 있지만, 동시에 여러개의 쓰레드를 깨워야할 필요도 있을것이다. 다음은 이러한 시그널을 보내는 함수들이다.**

**int pthread_cond_signal(pthread_cond_t *cond);**
**int pthread_cond_broadcast(pthread_cond_t *cond);**
				
pthread_cond_signal() 은 하나의 쓰레드를 깨우기 위한, pthread_cond_broadcast() 는 모든 쓰레드를 깨우기 위한 신호를 보내기 위해서 사용된다.
성공적으로 시그널이 전달되면 0이 리턴된다.

### 조건변수를 통한 신호 기다리기
신호를 보내는 쓰레드가 있다면, 이 신호를 받기 위해 기다리는 쓰레드도 있어야 할것이다. **신호를 받기 위해서는 pthread_cond_wait()와 pthread_cond_timewait() 두개의 함수**가 사용된다. 이두개의 함수는 신호가 전달될때까지 기다리게 된다.

**int pthread_cond_wait(pthread_cond_t *cond, pthread_mutex_t *mutex);**
**int pthread_cond_timedwait(pthread_cond_t *cond, pthread_mutex_t *mutex, const struct timespec *abstime);**
				
첫번째 아규먼트()는 신호를 기다리기 위한 "조건변수" 이며 2번째는 mutext 변수이다. 이 두개의 함수는 실행되면 mutext:::잠금()을 풀고(unlock) 신호를 기다린다. 그러다가 신호가 전달되면 뮤텍스 잠금을 얻은 다음 해당영역을 실행하게 된다.
**pthread_cond_wait() 와 pthread_cond_timedwait() 의 다른점은 timeout 을 기다리는지에 따라서 구별**된다. **pthread_cond_timedwait 는 기다림의 시간을 설정하여서 일정한 시간동안 시그널이 도착하지 않으면 ETIMEDOUT 값을 리턴**한다. 반면 **pthread_cond_wait 는 시그널이 도착할때까지 무한정** 기다린다.

지금까지의 설명한 내용의 이해를 돕기 위해 간단한 흐름도를 그려보았다. 아래의 **흐름도는 pthread_cond_wait()를 사용했을경우**다.

~~~
// 뮤텍스 와 조건변수를 초기화 한다.  
pthread_mutext_t     request_lock = PTHREAD_MUTEX_INITILIZER;
pthread_cond_t       producer_cv  = PTHREAD_COND_INITIALIZER;

// 임계 영역에 대한 뮤텍스 잠금을 얻는다. 
int rc;
rc = pthread_mutext(&mutext_a);
if (rc)
{
    perror("mutext lock error "); 
    phread_exit(NULL);
}

// 조건변수 producer_cv 에 대한 시그널을 기다린다.  
// wait 에 들어가기전에 mutext 는 request_lock에 대한 잠금을 
// 얻는다. 
mutext_rc1 = pthread_cond_wait(&producer_cv,&request_lock)
if
{
    // mutext 잠금이 이루어진다.  
    // 코드 실행
}

// 뮤텍스 잠금을 돌려준다. 
pthread_mutex_unlock(&request_lock);
~~~				
다음은 pthread_cond_timedwait()를 사용했을때의 흐름도이다. pthread_cond_wait()와 비교해서 시간설정을 위한부분이 들어가고, 타임아웃이 발생했을때 처리하는 부분이 들어가는걸 빼고는 나머지 부분은 동일하다.
~~~
...
struct timeval  now;
struct timespec timeout;
int    done;
...
rc = pthread_mutext(&mutext_a);
if(rc)
{
   ...
}

// timeout 시간 설정을 위해서 사용한다. 
gettimeofday(&now);
timeout.tv_sec = noew.tv_sec + 5
timeout.tv_nsec = now.tv_usec * 1000;

// timeout 시간동안 signal 을 기다린다. 
rc = pthread_cond_timedwait(&got_request, &request_mutext, &timeout);

// pthread_cond_timedwait 가 넘겨준 값을 이용해서 분기한다. 
switch(rc)
{
    // 시그널이 발생했을때.. 
    case 0:
        ...
        break;
    // timeout 동안 시그널이 발생하지 않았을경우
    case ETIMEDOUT:
        ...
        break;
    // 그밖의 경우(에러)
    default:  
        ...
        break;
}
pthread_mutext_unlock(&request_mutex);
~~~

### 조건 변수 없애기(Destory)
더이상 사용할 필요가 없는 조건변수는 없애주도록 하자. 이렇게 함으로써 시스템자원()의 낭비를 막을수 있다. 조건변수를 없애기 위해서는 pthread_cond_destroy() 함수를 이용한다. 조견변수를 없애기 전에 조건변수를 사용하는 쓰레드가 없음을 확인해야 한다.

~~~
// 조건변수 got_request 를 삭제한다. 리턴값이 EBUSY일경우는 
// 아직 사용하는 쓰레드가 있으서 조건변수 삭제에 실패했을경우이다. 
int rc = pthread_cond_destory(&got_request);
if (rc == EBUSY)
{
    ....
}
~~~

# Thread-Specific Data

## 쓰레드간 전역변수 사용의 문제점
일반적인 단일 쓰레드:::프로그래밍()을 할때, (비록 권장하지는 않지만) 전역변수()를 사용하는 경우가 종종 있을것이다. **전역변수를 씀으로써 그렇지 않을때 보다 코드를 훨씬 쉽게 기술**할수도 있기때문이다.

멀티쓰레드:::프로그램()에서도 이러한 전역 변수가 필요할때가 있다. 그런데 전역변수를 쓰는데 약간의 문제가 발생한다. 이유는 **모든 쓰레드가 전역변수를 공유해 버리기 때문에, 쓰레드별 전역변수를 사용할수 없기 때문**이다. **fork() 를 이용한 멀티:::프로세스() 모델에서는 프로세스간 전역변수 공유가 되지 않기 때문에, 각 프로세스 개별적인 전역변수를 사용할수 있는것과는 대조적**이다.

그럼 어떻게 해야지 쓰레드 개별적으로 전역변수를 사용하게 할수 있는지 알아보도록 하겠다.

## Thread-Specific Data
### Thread-Specific Data 의 개념
이러한 문제의 해결을 위해 등장한것이, **Thread-Specific Data (이하 TSD)**이다. 같은 전역변수가 서로 다른 메모리 영역을 가르키게 하는 방법을 사용하는데, **다른 메모리 영역의 주소를 가리키는 각각의 다른 key 를 사용함으로써, 쓰레드간 개별적인 전역변수를 엑세스하게끔 만들어준다.**

이것은 **간단한 데이타베이스 형태인 key-value 관계로 나타낼수 있는데, 각각의 key 에 이름을 부여하고 이 key 가 value 를 가리킨다. 이 value 는 메모리 영역**이 될것이다.

![Alt text](./thread/thread_7.png)
**위의 그림은 쓰레드 A, B, C각 하나의 전역변수인 Global_data 에 어떻게 자신만의 전역데이타를 읽고 쓸수 있는지에 대한 개념**을 나타낸것이다.

### TSD 블럭의 할당(생성)
**pthread_key_create() 함수를 이용해서 모든 쓰레드에서 효력을 가지는 새로운 key를 할당**받을수 있다. **최초에 key 를 만들게 되면 이 key 는 기본적으로 NULL을 가르키게 된다. 나중에 쓰레드는 이 key 값을 변경하게 됨으로써 자신이 원하는 메모리 영역을 참조**할수 있게 된다. 다음은 이 함수가 어떻게 사용되는지에 대한 간단한 예이다. pthread_key_t 는 unsigned int 형이다.

~~~
int rc;
pthread_key_t list_key;

// clean_list 는 사용자 정의 함수로, 나중에 데이타가 필요없을때 
// 데이타를 제거(free) 하기 위해서 필요한 함수이다. 
extern void* clean_list(void*); 

// key 를 생성하며 데이타 제거를 위한 함수를 지정해준다. 
// cleanup_list 함수는 NULL 로 지정될수도 있으며, 
// 이경우 나중에 key 를 삭제하더라도 데이터가 제거되지는 않는다. 
rc=pthread_key_create(&list_key, cleanup_list);
~~~				
**pthread_key_create()를 실행한후에 list_key 는 새로 만들어진 key 를 가르킨다. pthread_key_create 는 성공적으로 실행될경우 0을 반환하고 그렇지 않을경우에는 적당한 에러코드를 반환**하도록 되어있다. 생성할수 있는 key 의 숫자는 제한되어 있는데, 생성할수 있는 최대크기는 PTHREAD_KEYS_MAX 에 정의 되어 있다. 이값은 OS 마다 다를수 있으며 보통 1024 의 크기를 가진다.

### TSD 블럭 에 대한 Access
**key 를 만들었다면 이제 value 에 엑세스를 해야 한다. 엑세스를 위해서 pthread_getspecific()와 pthread_setspecific()두개의 함수가 제공**된다. **첫번째 함수는 key 에 대한 value를 가져오기 위해서 사용되며, 두번째 함수는 key 에 value 를 세팅하기 위해서 사용된다**. value 는 (void *)형으로 원하는 어떤 데이타라도 넘길수 있도록 되어있다. 다음은 간단한 사용 예이다.

~~~    
phtread_key_t a_key;
int rc;

int *p_num = (int *)malloc(sizeof(int));
(*p_num) = 4;

rc = pthread_setspecific(a_key, (void *)p_num);
...
// 만약 a_key 에 대해서 p_num 값을 가져오고 싶다면
{
   int *p_keyval = (int *)pthread_getspecific(a_key);

   if (p_keyval != NULL)
   {
      printf("value of 'a_key' is: %d\n", *p_keyval);
   }
}
~~~

### TSD 블럭 삭제
더이상 사용할 필요가 없는 **TSD 블럭은 pthread_key_delete()를 이용해서 삭제**한다. **주의 할것은 이것은 key 를 삭제하는 것이지 실제 데이타의 메모리를 삭제하지는 않는다는 것이다**. 데이타 메모리의 삭제는 pthread_key_create() 로 key 를 만들때 지정한 데이타 삭제 함수가 이용된다. 만약 데이타 삭제 함수가 NULL 이였다면 데이타 삭제는 일어나지 않을것이다.

**int rc = pthread_key_delete(key);**		

# Thread Pool
멀티 쓰레드 프로그램의 성능을 높이기 위해서 미리 쓰레드를 준비해서, 쓰레드를 할당하는, 쓰레드 풀 방식이 널리 사용됩니다. 비교적 구현이 간단하고, 다양한 응용이 가능하기 때문입니다.
예전에 리얼 타임 시그널을 이용한 Thread Pool구현을 다루었는데, 이번에는 조건변수()를 이용한 구현을 알아보도록 하겠습니다. 쓰레드 풀과 조건변수에 대한 일반적인 설명은 링크들을 참고하시기 바랍니다.

## 아이디어
지금 다중 접속 네트워크 프로그램을 만들려고 하고 있습니다. 이 네트워크 프로그램은 웹 서버처럼 연결과 종료가 빈번한 프로그램입니다. 처음엔 클라이언트 연결 당 하나의 쓰레드를 생성하는 방식을 사용했습니다. 그런데, 초당 연결 클라이언트의 수가 늘어나자 성능이 눈에 띄게 떨어지기 시작했습니다.
아무래도 쓰레드 생성에 너무 많은 비용이 소모되는 것 같군요. 그래서 스레드 풀 방식으로 바꾸기로 했습니다.
지정된 수 만큼 미리 자식 쓰레드를 만들어 놓고, 클라이언트 연결이 들어오면 노는 자식 쓰레드를 깨워서 작업을 할당하는 방식입니다.

![Alt text](./thread/thread_8.png)
빨간색 쓰레드는 작업중인 쓰레드이고, 노란색 쓰레드는 노는 쓰레드 입니다. accept() 함수로 클라이언트 연결을 가져오면, 노는 쓰레드 중 하나를 깨웁니다. 깨울 때, 클라이언트 연결 소켓과 작업에 필요한 정보를 함께 넘기는 방식 입니다.

## 조건변수로 기다리기
쓰레드 풀 구현에서 가장 중요한 요소 중 하나는 **"작업 지시가 있을 때까지 봉쇄되도록"**구현하는 것입니다. 여기에서는 조건변수를 이용하기로 했습니다. 자식 쓰레드는 pthread_cond_wait()함수로 부모 쓰레드가 깨울 때까지 기다립니다. **부모 쓰레드가 깨우는 시점은 accept()함수가 반환하는 시점으로, pthread_cond_signal()함수를 이용해서 자식 스레드를 깨웁니다.**
pthread_cond_wait 함수는 시그널이 있을 때까지, 봉쇄되므로 "작업지시가 있을 때까지 봉쇄돌 것"이라는 조건은 일단 만족을 시켰습니다.
또 하나 만족시켜야 할 조건으로 "봉쇄가 풀려서 작업중일 때는 이 자식 쓰레드를 호출하지 않도록 할 것"입니다. 이것도 간단하게 해결할 수 있습니다. 조건변수는 뮤텍스()와 함께 사용되기 때문이죠. **pthread_cond_wait 함수는 매개변수로 조건변수와 함께 뮤텍스를 사용합니다. _cond_wait함수를 호출하면 매개변수로 주어진 뮤텍스의 잠금을 자동으로 돌려**줍니다. **그러다가 시그널을 받고 반환하면, 뮤텍스 잠금을 얻게 됩니다.** 즉 작업영역을 임계영역으로 하는 거죠.
그러므로 부모 프로세스는 단지 자식 쓰레드의 뮤텍스 잠금을 조사하는 것만으로도 사용할 수 있는 뮤텍스인지 아닌지 확인할 수 있습니다. **pthread_mutex_trylock()함수로 바로 잠금을 얻고 진입이 가능하면, 놀고 있는 자식 쓰레드임을 의미**
**조건변수 하나로 "작업 지시가 있을 때까지 봉쇄", "작업 중 진입금지"까지 한번에 마무리**

## 자료 구조
이 자료구조는 부모 쓰레드와 자식 쓰레드가 공유합니다. 뮤텍스와 조건변수, 소켓정보등이 들어갑니다. 자식 쓰레드마다 각각의 뮤텍스와 조건변수, 소켓정보를 가져야 하므로 자식 쓰레드 갯수만큼 만들어서 관리해야 합니다.
자료구조는 대략 다음과 같을 겁니다.

~~~
class ThreadInfo
{
	private:
		pthread_cont_t mcond; 
		pthread_mutex_t mmutex;
		int msocket;
	public:
		ThreadInfo(pthread_cond_t, pthread_mutex_t);
		int TryLock();
        int Job();
		int UnLock();
};
~~~
- mcond : 스레드가 사용할 조건변수
- mmutex : 스레드가 사용할 뮤텍스
- TryLock() : 조건변수와 함께 사용할 뮤텍스 잠금을 얻기위한 메서드
- Job() : 자식 쓰레드가 수행할 코드를 가진 메서드
- UnLock() : 자식 쓰레드가 작업을 마치고 잠금을 되돌려주기 위해서 사용한다.
이 자료구조는 자식 쓰레드 갯수 만큼 생성해야 하므로 배열로 관리해야 합니다. STL() vector로 관리하기로 했습니다.
**vector<ThreadInfo> ThreadList;**

## 쓰레드 풀 프로세스
쓰레드 풀 프로세스는 다음과 같습니다. 네트워크 서버
1. 쓰레드 풀을 만든다.
2. 지정된 갯수 만큼의 쓰레드 풀을 만든다. 
3. 메인쓰레드와 자식쓰레드는 ThreadInfo 클래스로 조건변수와 뮤텍스를 공유한다. 
4. 자식 쓰레드는 조건변수와 뮤텍스를 초기화 하고, Job메서드를 실행한다. 
5. Job 메서드는 대략 다음과 같을 것이다.
~~~
Job()
{
	GetLock(&mutex);                         // 먼저 잠금을 얻는다.
	while(1)
	{
    	// 조건 변수로 기다린다. 조건 변수를 기다리면서 mutex 잠금을 내놓으면,
        // 부모 쓰레드는 mutex 잠금을 얻을 수 있는 상태가 되는데,
        // 이는 선택한 자식 쓰레드가 작업을 할 수 있는 상태를 의미한다. 
        pthread_cond_wait(&cond, &mutex);       
        // 부모 쓰레드가 cont_signal을 전송하면, _cond_wait는 반환하고
        // 잠금을 얻는다.  
    
        /*
         * 작업을 한다.
         */
   
        // 작업을 마치면, while 문 처음으로 가서 pthread_cond_wait를 호출한다. 
    }
}
~~~
6. 작업을 하는 동안은 자식 쓰레드()가 뮤텍스() 잠금을 얻은 상태이므로 부모는 TryLock 메서드를 호출하는 것으로 이 쓰레드가 작업 중임을 알 수 있다.
7. 작업을 마치면 다시 조건변수에서 기다린다.
8. 반복...

**이 방식은 자식 쓰레드가 잠금 권한을 가지고 실행하기 때문에, 동기화 문제에서 자유롭다는 장점**을 가진다. 자식 쓰레드는 단지 pthread_cond_wait를 호출 할 때만 잠금을 내놓는다.

## 코드
~~~
#include <pthread.h>
#include <iostream>
#include <vector>

#include <errno.h>

#define THREAD_POOL_SIZE 5

using namespace std;

class Thread
{
	private:
		pthread_mutex_t mmutex;
		pthread_cond_t mcond;
	public:
		int id;
		Thread();
		int GetLock();
		int TryLock();
		int UnLock();
		int Job();
		int Signal();
};

// 뮤텍스와 조건변수 초기화 
Thread::Thread()
{
	pthread_mutex_init(&mmutex, NULL);
	pthread_cond_init(&mcond, NULL);
}

// 뮤텍스 자금 얻기
int Thread::GetLock()
{
	return pthread_mutex_lock(&mmutex);
}

// 모텍스 잠금 얻기 시도
int Thread::TryLock()
{
	return pthread_mutex_trylock(&mmutex);
}

// 조건변수에 시그널을 전송한다.
// 시그널을 전송한 후에는 뮤텍스 잠금을 되돌려준다.
int Thread::Signal()
{
	pthread_cond_signal(&mcond);
	UnLock();
}

// 뮤텍스 잠금 되돌려준다.
int Thread::UnLock()
{
	pthread_mutex_unlock(&mmutex);
}

// 자식 쓰레드에서 실행할 작업 메서드
int Thread::Job()
{
	GetLock();
	while(1)
	{
		pthread_cond_wait(&mcond, &mmutex);
		// 임계 영역 -----------------------
        // 작업영역으로 자식쓰레드가 이 영역에 머물러 있을 때에는
        // 부모 쓰레드가 TryLock로 잠금을 얻지 못한다.
		cout << "Job Start " << id << endl;
		sleep(3);
		cout << "Job End " << id << endl;
        // ---------------------------------
	}
}

// 쓰레드 함수
void *thread_func(void *arg)
{
	Thread *lThread = (Thread *)arg;
	lThread->Job();
}

int main(int argc, char **argv)
{
	int i = 0;
	pthread_t p_thread;
	vector<Thread *> ThreadList;
	Thread *lThread;

	// 쓰레드 풀을 만든다.
	for(i = 0; i < THREAD_POOL_SIZE; i++)
	{
		lThread = new Thread();
		lThread->id = i+1;
		ThreadList.push_back(lThread);
		pthread_create(&p_thread, NULL, thread_func, (void *)lThread);
		usleep(100);
	}

	int mstat;
	while(1)
	{
		// 작업 가능한 쓰레드를 찾아서
		// 조건변수 시그널을 전송한다.
		for (i = 0; i < THREAD_POOL_SIZE; i++)
		{
			if((mstat = ThreadList[i]->TryLock()) == 0)
			{
				ThreadList[i]->Signal();
				break;
			}
		}
		sleep(1);
	}
}
~~~

## 문제
위 프로그램은 잘 돌아갈 것 같지만 몇 가지 문제점이 있다.
1. pthread mutex의 잠금은 잠금을 얻은 스레드에서 잠금을 되돌려줘야한다. 잠금을 다른 스레드에서 해제할 경우의 상황은 정의되어 있지 않다. 어떤 일이 발생할지 모른다.
일단 잠금을 얻은 스레드에서 잠금을 해제 하도록 수정해야 한다.
2. 조건변수를 제대로 사용하기 위해서는, 반드시 wait 함수로 먼저 기다려야 한다. 그러지 않을 경우 신호를 읽어 버린다. 그러므로 반드시 wait 함수가 먼저 호출 되도록, 즉 자식 스레드가 반드시 먼저 뮤텍스를 얻도록 장치를 마련해야 한다. flag 값을 검사하는 정도로 될 것 같다.
다음은 이러한 문제를 보강한 코드다. 테스트를 위한 코드도 추가했다.

~~~
#include <pthread.h>
#include <iostream>
#include <vector>
#include <stdio.h>
#include <string.h>

#include <errno.h>

#define THREAD_POOL_SIZE 5

using namespace std;

class Thread
{
	private:
		pthread_mutex_t mmutex;
		pthread_cond_t mcond;
		int value;
	public:
		int id;
		int LockFlag;
		Thread();
		int GetLock();
		int TryLock();
		int UnLock();
		int Job();
		int Signal();
		void SetValue(int a)
		{
			value = a;
		}
};

// 뮤텍스와 조건변수 초기화 
Thread::Thread()
{
	LockFlag = 1;
	value = 0;
	pthread_mutex_init(&mmutex, NULL);
	pthread_cond_init(&mcond, NULL);
}


// 모텍스 잠금 얻기 시도
int Thread::TryLock()
{
	int rtv;
	if(LockFlag != 1)
	{
		return -1;
	}
   	LockFlag = 0;
	rtv = pthread_mutex_lock(&mmutex);
	return 0;
}

// 조건변수에 시그널을 전송한다.
// 시그널을 전송한 후에는 뮤텍스 잠금을 되돌려준다.
int Thread::Signal()
{
	pthread_cond_signal(&mcond);
	UnLock();
}

// 자식 쓰레드에서 실행할 작업 메서드
int Thread::Job()
{
	char fname[30];
	FILE *fp;
	sprintf(fname, "Walk_%d.txt", id);
	fp = fopen(fname,"w");
	while(1)
	{
		pthread_mutex_lock(&mmutex);

		LockFlag = 1;		
		pthread_cond_wait(&mcond, &mmutex);
		LockFlag = 0;
		
		cout << "Walk " << id << endl;
		fputs("Job\n", fp);
		usleep(1000);
		pthread_mutex_unlock(&mmutex);
	}
}

// 뮤텍스 잠금 되돌려준다.
int Thread::UnLock()
{
	pthread_mutex_unlock(&mmutex);
}


// 쓰레드 함수
void *thread_func(void *arg)
{
	Thread *lThread = (Thread *)arg;
	lThread->Job();
}

int main(int argc, char **argv)
{
	int i = 0;
	int mstat;
	pthread_t p_thread;
	vector<Thread *> ThreadList;
	Thread *lThread;
	pthread_mutex_t mutex_lock;
	pthread_mutexattr_t attr;
	int kind;



	// 쓰레드 풀을 만든다.
	for(i = 0; i < THREAD_POOL_SIZE; i++)
	{
		lThread = new Thread();
		lThread->id = i+1;
		ThreadList.push_back(lThread);
		pthread_create(&p_thread, NULL, thread_func, (void *)lThread);
		usleep(100);
	}

	FILE *fp;
	fp = fopen("Job.txt","w");
	int j = 0;
	while(1)
	{
		// 작업 가능한 쓰레드를 찾아서
		// 조건변수 시그널을 전송한다.
		for (i = 0; i < THREAD_POOL_SIZE; i++)
		{
			if((mstat = ThreadList[i]->TryLock()) == 0)
			{
				cout << "Job Signal " << i + 1<< endl;
				fputs("Job Signal\n", fp);
				ThreadList[i]->SetValue(i+1);
				ThreadList[i]->Signal();
				break;
			}
		}
		j ++;
		if(j == 10000) break;
		usleep(10);
	}
}
~~~

[http://www.joinc.co.kr/w/Site/Thread#s-5.][http://www.joinc.co.kr/w/Site/Thread#s-5.]